In [1]:
library(ggplot2)
library(reshape2)
library(RColorBrewer)
suppressMessages(library(dplyr))
library(stringr)
suppressMessages(library(tidyr))
theme_set(theme_bw())
library(scales)
library(IRdisplay)
library(knitr)
options(repr.plot.width = 7, repr.plot.height = 4)
isotypes = c('Ala', 'Arg', 'Asn', 'Asp', 'Cys', 'Gln', 'Glu', 'Gly', 'His', 'Ile', 'iMet', 'Leu', 'Lys', 'Met', 'Phe', 'Pro', 'Ser', 'Thr', 'Trp', 'Tyr', 'Val')
single_positions = c('X8'='8', 'X9'='9', 'X14'='14', 'X15'='15', 'X16'='16', 'X17'='17', 'X17a'='17a', 'X17b'='17b', 'X18'='18', 'X19'='19', 'X20'='20', 'X20a'='20a', 'X20b'='20b', 'X21'='21', 'X26'='26', 'X32'='32', 'X33'='33', 'X34'='34', 'X35'='35', 'X36'='36', 'X37'='37', 'X38'='38', 'X44'='44', 'X45'='45', 'X46'='46', 'X47'='47', 'X48'='48', 'X54'='54', 'X55'='55', 'X56'='56', 'X57'='57', 'X58'='58', 'X59'='59', 'X60'='60', 'X73'='73')
paired_positions = c('X1.72'='1:72', 'X2.71'='2:71', 'X3.70'='3:70', 'X4.69'='4:69', 'X5.68'='5:68', 'X6.67'='6:67', 'X7.66'='7:66', 'X8.14'='*8:14', 'X10.25'='10:25', 'X11.24'='11:24', 'X12.23'='12:23', 'X13.22'='13:22', 'X15.48'='*15:48','X18.55'='*18:55', 'X19.56'='*19:56', 'X27.43'='27:43', 'X28.42'='28:42', 'X29.41'='29:41', 'X30.40'='30:40', 'X31.39'='31:39', 'X49.65'='49:65', 'X50.64'='50:64', 'X51.63'='51:63', 'X52.62'='52:62', 'X53.61'='53:61', 'X54.58'='*54:58')
fills = c('A'='#ffd92f', 'C'='#4daf4a', 'G'='#e41a1c', 'U'='#377eb8', 'A:U'='#93da69', 'U:A'='#93da69', 'G:C'='#c1764a', 'C:G'='#c1764a', 'G:U'='#b26cbd', 'U:G'='#b26cbd', '-'='gray60', '-:-'='gray60')
suppressWarnings(suppressMessages(library(Biostrings)))
options(knitr.table.format = 'html')

In [2]:
paired_positions = c('X1.72'='1:72', 'X2.71'='2:71', 'X3.70'='3:70', 'X4.69'='4:69', 'X5.68'='5:68', 'X6.67'='6:67', 'X7.66'='7:66', 'X8.14'='*8:14', 'X10.25'='10:25', 'X11.24'='11:24', 'X12.23'='12:23', 'X13.22'='13:22', 'X15.48'='*15:48','X18.55'='*18:55', 'X19.56'='*19:56', 'X27.43'='27:43', 'X28.42'='28:42', 'X29.41'='29:41', 'X30.40'='30:40', 'X31.39'='31:39', 'X49.65'='49:65', 'X50.64'='50:64', 'X51.63'='51:63', 'X52.62'='52:62', 'X53.61'='53:61', 'X54.58'='*54:58')

In [3]:
load('best-freqs.RData')
load('clade-isotype-specific.RData')
load('isotype-specific.RData')
load('consensus-IDEs.RData')
load('clade-isotype-specific-freqs.RData')

In [123]:
identities = read.delim('identities.tsv', sep='\t')
identities_all = identities
identities$quality = as.logical(identities$quality)
identities$restrict = as.logical(identities$restrict)
identities = identities %>% filter(!restrict, !(isotype %in% c('SeC', 'Sup')))

In [5]:
genome_table = read.delim('genome_table+.txt', sep='\t', stringsAsFactors=FALSE, header=FALSE, col.names=c("species_short", "species", "species_long", "domain", "clade"))

# Introduction

Previous analysis in `euk-tRNAs` and `isotype-clade-specific` focused on processing and dicing the data with appropriate assumptions. Now, we move on to the meat of this project. For **new identity stories**, I'll employ a few strategies to find interesting stories in a systematic way. Some of these will have been covered in `first-pass-consensus`.

1. Find rejected universal sequence features. Set a cutoff at 95% presence, but fails (a) isotype check, (b) clade check, or (c) species check. Spin in previous work, if any. Bonus points if previous work _conflicts_ with our frequencies. I will select one of each for inclusion in the paper.
    
2. Find rejected isotype-specific sequence features. Slightly different from (1a) above, since this could apply to highly isotype-specific positions (e.g. N73). For the paper, find one example (choose between species or clade checks).

Per Todd's advice, I'll take a look at a few in each category.

In [124]:
# No. tRNAs we're looking at
identities %>% nrow
identities_all %>% nrow

[1] 25517

[1] 76160

# Strategy 1: Universal features

### Find targets

In [6]:
code_groups = c('A'=1, 'C'=1, 'G'=1, 'U'=1, 'Absent'=1, 
                'Purine'=2, 'Pyrimidine'=2,
                'Weak'=3, 'Strong'=3, 'Amino'=3, 'Keto'=3,
                'B'=4, 'D'=4, 'H'=4, 'V'=4,
                'GC'=1, 'AU'=1, 'UA'=1, 'CG'=1, 'GU'=1, 'UG'=1,
                'StrongPair'=2, 'WeakPair'=2, 'Wobble'=2,
                'PurinePyrimidine'=3, 'PyrimidinePurine'=3, 'AminoKeto'=3, 'KetoAmino'=3,
                'Paired'=4, 'Mismatched'=4, 'Bulge'=4)

cutoff_freqs = data.frame()
for (cutoff in c(0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99, 1.0)) {
  df = clade_iso_ac_freqs %>%
    group_by(positions, variable) %>%
    summarize(count=sum(value), freq=sum(value)/sum(total)) %>%
    filter(freq >= cutoff) %>%
    mutate(cutoff=as.character(cutoff)) %>%
    select(positions, variable, freq, cutoff) %>%
    group_by(positions) %>%
    arrange(code_groups[variable], desc(freq)) %>%
    filter(row_number(positions) == 1)
  if (nrow(cutoff_freqs) == 0) cutoff_freqs = df
  else cutoff_freqs = rbind(cutoff_freqs, df)
}

In [43]:
df = cutoff_freqs %>% 
  mutate(variable = as.character(variable)) %>%
  bind_rows(consensus %>% mutate(variable = identity, freq = 0.9, cutoff = "Consensus") %>% select(positions, variable, freq, cutoff)) %>%
  filter(positions %in% c(names(single_positions), names(paired_positions))) %>%
  select(positions, cutoff, variable) %>%
  group_by(positions) %>%
  spread(cutoff, variable)

df[which((df$`0.9` != df$Consensus) | (df$`0.99` != df$Consensus) | (is.na(df$Consensus) & df$`0.9` %in% c('A', 'C', 'G', 'U', 'AU', 'GC', 'CG', 'UA', 'GU', 'UG', 'Purine', 'Pyrimidine' ,'PurinePyrimidine', 'PyrimidinePurine'))) , ]

,positions,0.5,0.6,0.7,0.8,0.9,0.95,0.99,Consensus
1,X11.24,CG,CG,CG,PyrimidinePurine,PyrimidinePurine,PyrimidinePurine,Paired,Paired
2,X15,G,G,G,G,Purine,Purine,Purine,NA
3,X15.48,GC,GC,GC,GC,PurinePyrimidine,Paired,NA,NA
4,X16,U,U,U,U,Pyrimidine,B,NA,NA
5,X18,G,G,G,G,G,G,G,V
6,X18.55,GU,GU,GU,GU,GU,GU,Paired,NA
7,X19.56,GC,GC,GC,GC,GC,GC,Paired,GC
8,X21,A,A,A,A,A,A,A,NA
9,X32,C,C,C,Pyrimidine,Pyrimidine,Pyrimidine,B,B
10,X33,U,U,U,U,U,U,Pyrimidine,Pyrimidine


For strategy 1A, some good candidates here: 
- R9 (95%)
- Y11:R24 (99%)
- G18, U55, and G18:U55 (99%)
- G19:C56 (99%)
- A21 and A14/A21 (99%)
- U33 (99%)
- R37 (99%)
- R46 (99%)
- G53:C61 (99%)
- U54:A58 (99%)
- U55 (99%)
- R57 (99%)
- Y60 (90%)

### R9 and R46

#### What's known

- Marck and Grosjean have a purine for iMet and no consensus for elongator, though they are mostly R or V. R9 is also conserved in initiators for archaea/bacteria.
- 9-12-23 is a tertiary interaction. [Gautheret et al. (1995)](http://dx.doi.org/10.1006/jmbi.1995.0200) has these frequencies from the Sprinzl 1991 database: ![9-12-23 frequency matrix](figures/9-12-23-gautheret.png)

- Position 9 is known to be modified with m$^1$G (along with 37) in a wide range of eukaryotes. 

#### Our data

- Here's our frequencies:

In [22]:
table(paste0(identities$X12, ':', identities$X23), identities$X9)

     
          -     A     C     G     U
  A:A     0    10     1     8     0
  A:C     0    27     2   243     0
  A:G     0     2     1    15     0
  A:U     0   599   153    71     4
  C:-     0     0     0     1     0
  C:A     0    14     0    97     1
  C:C     0     1     0     5     0
  C:G     0   312    99  7566    66
  C:U     0     0     0     6     0
  G:A     0     3     0     3     0
  G:C     1  1782     8  7787     7
  G:G     0     0     0     8     0
  G:U     0    16     1    46     0
  K:A     0     1     0     0     0
  R:C     0     0     0     1     0
  U:A     1 10436     7  1940    44
  U:C     0     5     0     3     0
  U:G     0    56     0    76     0
  U:M     0     1     0     0     0
  U:R     0     1     0     0     0
  U:U     0    11     0     1     0
  U:W     0     1     0     0     0

**Which isotypes/clades/species fail the consensus checks?**

In [23]:
## Clade/isotype check
best_freqs %>% filter(positions == 'X9') %>% group_by(clade, isotype) %>%
  summarize(status=sum((variable %in% c("G", "A", "Purine"))*count)/sum(count) == 1) %>%
  filter(!status)

## Species check
identities %>% select(clade, species, isotype, X9) %>% group_by(clade, species, isotype) %>%
  summarize(status=sum(X9 %in% c("G", "A", "Purine")/n()) >= 0.1,
            freq=sum(X9 %in% c("G", "A", "Purine")), ntRNAs=n()) %>%
  filter(!status)

,clade,isotype,status
1,Insecta,Glu,FALSE
2,Insecta,His,FALSE
3,Mammalia,His,FALSE
4,Vertebrata,His,FALSE


,clade,species,isotype,status,freq,ntRNAs
1,Fungi,crypGatt_WM276,Gly,FALSE,0,1
2,Fungi,crypNeof_VAR_GRUBII_H99,Gly,FALSE,0,1
3,Fungi,crypNeof_VAR_NEOFORMANS_B_3501,Gly,FALSE,0,1
4,Fungi,crypNeof_VAR_NEOFORMANS_JEC21,Gly,FALSE,0,1
5,Insecta,aedAeg1,His,FALSE,0,5
6,Insecta,apiMel1,His,FALSE,0,1
7,Insecta,bomTer1,His,FALSE,0,2
8,Insecta,dm6,His,FALSE,0,1
9,Insecta,dp4,His,FALSE,0,1
10,Insecta,droAna3,His,FALSE,0,1


#### What's new

The 9:12:23 ratios are a bit different. But it shows the same thing: there is some selectivity for what base triples are allowed, but enough tolerance that there are a few interactions that persist.

The default hypothesis is that the 9-23 interaction has a conserved interaction type that explains the frequencies shown. We can marginalize as such because 12:23 is always a WC pair. This is not the case - although some (R:R) have a a strong trans WC-Hoogsteen pair of hydrogen bonds, others (C:R) do not. The hoogsteen side of C is just two stable carbons unlikely to hydrogen bond.

Examining the frequencies _without_ histidine shows that the C9:U23 is conserved in His, and A9:U23 is conserved in Asp, for some clades. (See tertiary interactions figure).

In [24]:
table(identities$X9, identities$X23)

   
        -     A     C     G     M     R     U     W
  -     0     1     1     0     0     0     0     0
  A     0 10464  1815   370     1     1   626     1
  C     0     8    10   100     0     0   154     0
  G     1  2048  8039  7665     0     0   124     0
  U     0    45     7    66     0     0     4     0

So here's what we know.
- 12:23 is almost always WC paired, so we can marginalize it.
- 9 is typically a purine, though there are isotype- and clade-specific exceptions. For example, His and Asp deviate in 4 clades.
    - In the His case, C and U can interact via the 2-carbonyl [amino pairing](http://www.columbuslabs.org/wp-content/uploads/2008/03/basepairs.pdf).
- 9:23 is thought to be a _trans_ interaction with two hydrogen bonds.

**Conclusion 1**: Position 9 was thought to be a purine. Instead, it varies by isotype and clade. Much of the variation from R9 can be explained by looking at 9:23 as an clade/isotype discriminating tertiary interaction.

We can leave it at that. But we should also look into compensatory interactions if 9:23 is disrupted.

### Covariation for core D stem 3d pairs

#### tRNA covariation frequencies for 9:23, 22:46, and 10:45

In [25]:
df = table(paste0(identities$X9, ':', identities$X23),
           paste0(identities$X46, ':', identities$X22, ' / ', identities$X45, ':', identities$X10),
           identities$isotype)

In [26]:
as.data.frame(df) %>% group_by(Var3) %>% filter(Freq > 50)

,Var1,Var2,Var3,Freq
1,A:A,G:A / -:G,Ala,1328
2,A:A,G:G / A:G,Ala,71
3,A:A,G:G / G:G,Ala,2005
4,G:A,G:G / G:G,Ala,75
5,G:C,G:G / G:G,Ala,95
6,G:A,G:U / G:G,Ala,51
7,G:C,A:A / G:G,Arg,315
8,G:G,A:A / G:G,Arg,1106
9,A:A,G:G / G:G,Arg,75
10,A:C,G:G / G:G,Arg,56


This is messy. For each isotype there may be a different tertiary interaction compensating for the lack of A:A. It seems that purine:purine interactions are unusually enriched among all three of these except in valine. 

![classical interaction structure](figures/3d-interactions-oliva.png)

#### Average number of purine:purine interactions by isotype and clade

In [27]:
RRs = c("A:A", "A:G", "G:G", "G:A")
identities %>% select(isotype, clade, X9.23, X22.46, X10.45, X26.44) %>% 
  rowwise() %>% 
  mutate(nRR=(X9.23 %in% RRs) + (X22.46 %in% RRs) + (X10.45 %in% RRs) + (X26.44 %in% RRs)) %>%
  group_by(isotype, clade) %>% 
  summarize(nRR=signif(mean(nRR), 3)) %>%
  spread(isotype, nRR)

Warning message:
“Grouping rowwise data frame strips rowwise nature”

,clade,Ala,Arg,Asn,Asp,Cys,Gln,Glu,Gly,His,⋯,Leu,Lys,Met,Phe,Pro,Ser,Thr,Trp,Tyr,Val
1,Fungi,2.88,3.16,3.45,3.05,3.19,1.47,1.97,1.96,2.83,⋯,1.97,3.33,3.61,3.86,2.03,1.78,3.8,3.06,3.3,2.33
2,Insecta,3.04,3.6,3.3,2.16,3.86,1.98,3.4,2.64,2.03,⋯,2.02,3.87,3,3.17,2.13,2.32,3.89,2.88,3.89,2.17
3,Mammalia,3.93,3.53,2.93,2.16,3.97,1.97,2.96,2.64,2.04,⋯,2,3.92,2.99,3.93,2,2.05,3.99,2.98,3.99,2.19
4,Nematoda,3.99,3.55,2.97,2,3.96,1.91,2.79,2.46,1.96,⋯,1.97,2.82,3.04,3.86,2.01,2.56,3.96,2.9,3.96,2
5,Spermatophyta,3.98,3.46,3.75,2.21,3.71,2.45,3.46,3.34,2.95,⋯,1.96,3.02,3,3.79,2.06,1.24,3.71,3,3.82,2
6,Vertebrata,3.96,3.44,2.93,2.05,3.89,1.98,2.96,2.34,1.99,⋯,1.97,3.94,2.96,3.71,2,2.05,3.95,2.99,3.92,1.97


#### Average number of purine:purine interactions by position/isotype

In [30]:
RRs = c("A:A", "A:G", "G:G", "G:A")
identities %>% select(isotype, X9.23, X22.46, X10.45, X26.44) %>% 
  rowwise() %>% 
  mutate(X9.23=X9.23 %in% RRs, X22.46=X22.46 %in% RRs, X10.45=X10.45 %in% RRs, X26.44=X26.44 %in% RRs) %>%
  gather(position, RR, X9.23, X22.46, X10.45, X26.44, -isotype) %>%
  group_by(isotype, position) %>%
  summarize(RR=round(mean(RR), digits=1)) %>%
  spread(isotype, RR)

,position,Ala,Arg,Asn,Asp,Cys,Gln,Glu,Gly,His,⋯,Leu,Lys,Met,Phe,Pro,Ser,Thr,Trp,Tyr,Val
1,X10.45,0.6,1,1,1,1,1,0.9,0.9,1,⋯,0,0.9,1,0.9,1,0.1,1,1,1,0.9
2,X22.46,1,1,1,1,1,0.8,0.9,0.9,0.9,⋯,1,1,1,1,0,1,1,1,0.9,0.1
3,X26.44,1,0.9,0.9,0.2,0.9,0,0.3,0.1,0.2,⋯,0,0.8,1,1,0,0.6,1,0.9,1,1
4,X9.23,1,0.6,0.3,0.2,1,0.1,0.8,0.6,0.2,⋯,1,0.9,0.2,0.9,1,0.4,1,0.2,0.9,0.1


#### Average number of interactions containing a purine by position/isotype

In [31]:
RRs = c("A:A", "A:G", "G:G", "G:A", "G:C", "C:G", "A:C", "C:A", "U:G", "G:U", "A:U", "U:A")
identities %>% select(isotype, X9.23, X22.46, X10.45, X26.44) %>% 
  rowwise() %>% 
  mutate(X9.23=X9.23 %in% RRs, X22.46=X22.46 %in% RRs, X10.45=X10.45 %in% RRs, X26.44=X26.44 %in% RRs) %>%
  gather(position, RR, X9.23, X22.46, X10.45, X26.44, -isotype) %>%
  group_by(isotype, position) %>%
  summarize(RR=round(mean(RR), digits=1)) %>%
  spread(isotype, RR)

,position,Ala,Arg,Asn,Asp,Cys,Gln,Glu,Gly,His,⋯,Leu,Lys,Met,Phe,Pro,Ser,Thr,Trp,Tyr,Val
1,X10.45,0.6,1,1,1,1,1,1,0.9,1,⋯,0,1,1,1,1,0.2,1,1,1,1
2,X22.46,1,1,1,1,1,1,1,1,1,⋯,1,1,1,1,1,1,1,1,1,1
3,X26.44,1,1,1,0.8,1,0.2,0.4,1,0.4,⋯,1,1,1,1,1,1,1,1,1,1
4,X9.23,1,1,1,1,1,1,1,1,0.5,⋯,1,1,1,1,1,1,1,1,1,1


#### Are non-purine-containing interactions enriched for some other type of interaction?

In [32]:
identities %>% select(isotype, X9.23, X22.46, X10.45, X26.44) %>% 
  gather(position, identity, X9.23, X22.46, X10.45, X26.44, -isotype) %>%
  filter(!(identity %in% c(RRs, "G:-", "A:-"))) %>%
  group_by(position, isotype, identity) %>%
  summarize(count=n()) %>%
  filter(count > 20) %>%
  spread(identity, count, fill=0)

Warning message:
“attributes are not identical across measure variables; they will be dropped”

,position,isotype,C:-,C:U,U:-,U:C,U:U
1,X10.45,Leu,71,0,0,0,0
2,X10.45,Ser,139,0,239,0,0
3,X26.44,Asp,0,0,0,0,79
4,X26.44,Gln,0,35,0,754,129
5,X26.44,Glu,0,0,0,569,102
6,X26.44,Gly,0,0,0,0,29
7,X26.44,His,0,26,0,184,0
8,X26.44,Ile,0,0,0,33,0
9,X9.23,His,0,153,0,0,0


They are not - but they are isotype-specific.

**Summary**: Tertiary interactions within this "tRNA core region" along the D stem are enriched in purines and purine-purine interactions, though the extent varies by isotype and position. Exceptions are isotype-specific.

### G18, U55, and G18:U55

#### Number of tRNAs without G18:U55

In [33]:
identities %>% select(clade, species, X18, X55, X18.55) %>%
  filter(X18.55 != "G:U") %>% nrow()

[1] 405

#### Which isotypes/clades/species fail the consensus checks?

In [34]:
## Clade/isotype check
best_freqs %>% filter(positions == 'X18.55') %>% group_by(clade, isotype) %>%
  summarize(status=sum((variable == "GU")*count)/sum(count) == 1) %>%
  filter(!status)

## Species check
identities %>% select(clade, species, isotype, X18.55) %>%
  group_by(clade, species, isotype) %>%
  summarize(status=sum((X18.55 == "G:U")/n()) >= 0.1,
            freq=sum(X18.55 == "G:U"), tRNAs=n()) %>%
  filter(!status)

,clade,isotype,status


,clade,species,isotype,status,freq,tRNAs
1,Fungi,flamVelu_KACC42780,Pro,0,0,1
2,Fungi,valsMali_03_8,Pro,0,0,2


Earlier, we had a vertebrate check fail because of the methods used to build the quality set. I've relaxed the species check constraints to 10% to better identify IDEs even when there are massively amplified tRNAs. Curiously, in danRer10, gasAcu1, and oryLat2, the majority of the non-G:U tRNAs contain a G:C. It's pretty likely that, given the high copy number, many of these tRNAs are actually pseudogenes. In addition, there are a decent amount of G18:U55 tRNAs to actually drive translation.

The two fungi listed here are rare exceptions to the rule. _Flammulina velutipes_ is the enokitake mushroom, and none of its proline tRNAs contain G18. _Valsa mali_ is a fungus that infects apple trees. It contains 4 proline tRNAs. The first two (Pro-AGG-1, Pro-CGG-1) score at 22 and 47, contain G18:U55, and are filtered out. The other two (Pro-TGG-1) score at 58, and contain C18:U55. So we have two main possibilities here.

1. G18:U55 is required for proper folding and recognition. Pro-CGG-1 is used as the main proline tRNA.
2. G18:U55 is not required. The synthetase is able to recognize some combination of Pro-CGG and Pro-TGG.

The evidence below points to option 2. In fungi, TGG is a much more common anticodon than CGG. Pro-TGG in this species also scores better, and contains a position 47. There are only two other fungal proline tRNAs without a position 47, both of which exist in _Encephalitozoon_ species, and both of which are supplemented by other high-scoring proline tRNAs.

#### Fungal proline anticodon usage /  Fungal proline position 47 incidence

In [37]:
identities %>% select(clade, isotype, anticodon) %>%
  filter(isotype == "Pro" & clade == "Fungi") %>%
  group_by(clade, isotype, anticodon) %>%
  summarize(count=n())

identities %>% select(clade, isotype, X47) %>%
  filter(isotype == "Pro" & clade == "Fungi") %>%
  group_by(clade, isotype, X47) %>%
  summarize(count=n()) %>%
  spread(X47, count)

identities %>% select(clade, species, species_long, seqname, isotype, score, X47) %>%
  filter(isotype == "Pro" & clade == "Fungi" & X47 == "-")

,clade,isotype,anticodon,count
1,Fungi,Pro,AGG,65
2,Fungi,Pro,CGG,33
3,Fungi,Pro,TGG,92


,clade,isotype,-,C,U
1,Fungi,Pro,2,53,135


,clade,species,species_long,seqname,isotype,score,X47
1,Fungi,enceHell_ATCC50504,Encephalitozoon hellem ATCC 50504,enceHell_ATCC50504_chrX.trna2-ProCGG,Pro,60.7,-
2,Fungi,enceInte_ATCC50506,Encephalitozoon intestinalis ATCC 50506,enceInte_ATCC50506_chrX.trna2-ProCGG,Pro,63.4,-


### G19:C56

We now look at 19:56 in context of 18:55. G19:C56 similarly hits 99% frequency.

#### Distribution of 18:55 and 19:56

In [38]:
identities %>% select(X18.55, X19.56) %>%
  group_by(X18.55, X19.56) %>%
  summarize(count=n()) %>%
  filter(count > 5) %>%
  spread(X19.56, count, fill=0)

,X18.55,A:C,A:U,C:C,G:A,G:C,G:G,G:U,U:C
1,A:U,0,0,0,0,182,0,11,0
2,C:U,0,0,0,0,28,0,0,0
3,G:A,0,0,0,0,10,0,0,0
4,G:C,0,0,0,0,109,0,0,0
5,G:G,0,0,0,0,15,0,0,0
6,G:U,130,6,20,61,30302,38,545,30
7,U:U,0,0,0,0,29,0,0,0


What may be happening here is that 18:55 and 19:56 act in concert. Both are required, but failing that, having at least one of them will weakly preserve the tertiary structure. This makes sense in context of the tertiary structure shown above - 18:55 and 19:56 are right next to each other and perform the same function, forming the tRNA elbow. 

In this case, whether it fails the species or isotype/clade check doesn't matter anymore, unless we specifically see enrichment for a clade/isotype.

The one exception is with the 9 tRNAs for A18:U55 and G19:U56. Let's take a look at both of them.

#### Is there enrichment for certain isotypes/clades for the non-GU/GC tRNAs?

This measures the percentage of tRNAs in the isotype/clade with non-GU/GC tRNAs.

**Either no G:U or no G:C:**

In [49]:
identities %>% select(isotype, clade, X18.55, X19.56) %>%
  group_by(isotype, clade) %>%
  mutate(total=n()) %>%
  filter(X18.55 != 'G:U' | X19.56 != 'G:C') %>%
  summarize(freq=signif(n()/unique(total), digits=2), count=n(), total=total[1]) %>%
  filter(freq > 0.05) %>%
  arrange(desc(freq))

,isotype,clade,freq,count,total
1,Gly,Vertebrata,0.21,54,253
2,Lys,Mammalia,0.17,127,739
3,Thr,Vertebrata,0.13,49,383
4,Met,Vertebrata,0.12,17,139
5,Glu,Nematoda,0.11,9,81
6,Lys,Vertebrata,0.11,31,294
7,Tyr,Vertebrata,0.11,21,191
8,Lys,Spermatophyta,0.098,21,214
9,Phe,Mammalia,0.096,22,228
10,Ile,Spermatophyta,0.09,16,178


**Neither G:U nor G:C:**

In [50]:
identities %>% select(isotype, clade, X18.55, X19.56) %>%
  group_by(isotype, clade) %>%
  mutate(total=n()) %>%
  filter(X18.55 != 'G:U' & X19.56 != 'G:C') %>%
  summarize(freq=signif(n()/unique(total), digits=2), count=n(), total=total[1]) %>%
  filter(freq > 0.01) %>%
  arrange(desc(freq))

,isotype,clade,freq,count,total
1,Asn,Insecta,0.018,1,57
2,Met,Fungi,0.015,2,130
3,Gly,Vertebrata,0.012,3,253
4,Lys,Mammalia,0.011,8,739


We see differential enrichment in the "not GU or not GC" case, but not in this case ("not GU/GC"). Which is to be expected - it's easier to mutate one or the other, rather than both. But just in case, we'll double check these clades.

#### Which species have which divergent identities at 18:55 and 19:56?

In [54]:
identities %>% select(isotype, clade, X18.55, X19.56) %>%
  filter((isotype == "Gly" & clade == "Vertebrata") | (isotype == "Lys" & clade == "Mammalia") | (isotype == "Met" & clade == "Fungi")) %>%
  group_by(isotype, clade, X18.55, X19.56) %>%
  summarize(count=n()) %>%
  filter(count > 1) %>%
  spread(X19.56, count, fill=0)

identities %>% select(isotype, clade, species, X18.55, X19.56) %>%
  filter(isotype == "Met" & clade == "Fungi") %>%
  group_by(species, clade, isotype) %>% mutate(total_tRNAs=n()) %>%
  filter(X18.55 == "G:-" & X19.56 == "G:-") %>%
  group_by(species, clade, isotype, total_tRNAs) %>% summarize(count=n())

identities %>% select(isotype, clade, species, X18.55, X19.56) %>%
  filter(isotype == "Gly" & clade == "Vertebrata") %>%
  group_by(species, clade, isotype) %>% mutate(total_tRNAs=n()) %>%
  filter(X18.55 == "G:C" & X19.56 == "G:C") %>%
  group_by(species, clade, isotype, total_tRNAs) %>% summarize(count=n())

identities %>% select(isotype, clade, species, X18.55, X19.56) %>%
  filter(isotype == "Lys" & clade == "Mammalia") %>%
  group_by(species, clade, isotype) %>% mutate(total_tRNAs=n()) %>%
  filter(X18.55 == "G:U" & X19.56 == "G:U") %>%
  group_by(species, clade, isotype, total_tRNAs) %>% summarize(count=n())

,isotype,clade,X18.55,A:C,A:U,G:-,G:A,G:C,G:G,G:U
1,Gly,Vertebrata,G:C,0,0,0,0,34,0,0
2,Gly,Vertebrata,G:U,0,0,0,0,199,0,14
3,Lys,Mammalia,A:U,0,0,0,0,48,0,2
4,Lys,Mammalia,C:U,2,0,0,0,4,0,0
5,Lys,Mammalia,G:U,11,2,0,2,612,2,47
6,Lys,Mammalia,U:U,0,0,0,0,3,0,0
7,Met,Fungi,G:-,0,0,2,0,0,0,0
8,Met,Fungi,G:U,0,0,0,0,128,0,0


,species,clade,isotype,total_tRNAs,count
1,fusaPseu_CS3096,Fungi,Met,4,1
2,fusaPseu_CS3270,Fungi,Met,4,1


,species,clade,isotype,total_tRNAs,count
1,danRer10,Vertebrata,Gly,53,33
2,fr3,Vertebrata,Gly,9,1


,species,clade,isotype,total_tRNAs,count
1,ailMel1,Mammalia,Lys,40,5
2,calJac3,Mammalia,Lys,8,1
3,canFam3,Mammalia,Lys,31,4
4,cavPor3,Mammalia,Lys,14,1
5,criGri1,Mammalia,Lys,39,2
6,dasNov3,Mammalia,Lys,29,2
7,eriEur2,Mammalia,Lys,50,7
8,felCat5,Mammalia,Lys,40,2
9,gorGor3,Mammalia,Lys,11,1
10,hetGla2,Mammalia,Lys,10,2


If it's the case that G:C/G:C or G:U/G:U is deficient, both vertebrate glycine and mammalian lysine are well-compensated in each species by other tRNAs, since there are so many tRNA copies.

Now to go back to the original 18:55 and 19:56 table - 

#### Is there enrichment in certain isotypes/clades/species for A18:U55 and G19:U56?

In [55]:
identities %>% select(isotype, clade, species, X18.55, X19.56) %>%
  group_by(isotype, clade, species) %>% mutate(total_tRNAs=n()) %>%
  filter(X18.55 == "A:U" & X19.56 == "G:U")

,isotype,clade,species,X18.55,X19.56,total_tRNAs
1,Tyr,Spermatophyta,carPap1,A:U,G:U,7
2,Lys,Mammalia,criGri1,A:U,G:U,39
3,Ala,Mammalia,hetGla2,A:U,G:U,26
4,Lys,Vertebrata,melUnd1,A:U,G:U,9
5,Gly,Mammalia,micMur1,A:U,G:U,8
6,Leu,Mammalia,nomLeu3,A:U,G:U,18
7,Lys,Vertebrata,oreNil2,A:U,G:U,18
8,Lys,Mammalia,otoGar3,A:U,G:U,11
9,Ala,Mammalia,speTri2,A:U,G:U,84
10,Cys,Mammalia,triMan1,A:U,G:U,19


There is slight enrichment, but it's nothing especially noteworthy, unless it's a conserved atypical tRNA. Let's check for that later. Each of these species has a lot of tRNAs that can compensate for one faulty tRNA. It's also worth noting that A:U and G:U are the most common single mutations, making it likely that they can still participate in the 3D interaction. Thus, a double mutation may be the strongest non-GU/GC interaction.

**Do the fungal proline tRNAs without G18:U55 have G19:C56?** They have to, otherwise they would've appeared in the table above!

#### What's known

G19 often contains a 2'-O-methyl modification, and in the majority of tested tRNAs, U55 is modified to $\Psi$. It's known that G18:U55 and G19:C56 are conserved.

A kinetics study showed an 80-fold decrease in translation efficiency (because of ribosome docking) without G18:U55 ([Pan et al. 2006](http://www.nature.com/nsmb/journal/v13/n4/full/nsmb1074.html)).

#### Summary

- 18:55 and 19:56 frequencies have been studied in isolation but not together.
- G18:U55 and G19:C56 are vital for forming the tertiary structure at the tRNA elbow, and exist in 96% of tRNAs.
- If it's not 100% conserved, is it a clade/isotype IDE, or is it an artifact of high-copy pseudogenes?
    - Most of the remaining 4% has one mutation. For the two most egregious isotypes/clades, each species has several more tRNAs of the same isoacceptor to compensate. These can be explained by pseudogenes getting through the quality filter.
    - There are also 9 tRNAs with two mutations. All of them are AU/GU, and like before, each species have several more tRNAs of the same isoacceptor to compensate.
- Based on this evidence, I would say that having one of G18:U55 and G19:C56 is an identity element.    

### Y11:R24

This one meets our definition of consensus, though it doesn't reach the 99% threshold. Let's see if the tRNAs themselves are enriched in any way.

#### Number of tRNAs without C11:G24 or U11:A24

In [57]:
identities %>% select(clade, species, X11.24) %>%
  filter(!(X11.24 %in% c("C:G", "U:A"))) %>% nrow()

[1] 582

#### Clade / isotype enrichment for tRNAs without Y11:R24

In [58]:
display_markdown(paste(as.character(kable(identities %>% select(clade, quality, isotype, X11.24) %>%
  filter(quality & !(X11.24 %in% c("C:G", "U:A"))) %>%
  group_by(clade, isotype) %>%
  summarize(count = n()) %>%
  spread(isotype, count, fill = 0))), collapse = "\n"))

<table>
 <thead>
  <tr>
   <th style="text-align:left;"> clade </th>
   <th style="text-align:right;"> Ala </th>
   <th style="text-align:right;"> Arg </th>
   <th style="text-align:right;"> Asn </th>
   <th style="text-align:right;"> Asp </th>
   <th style="text-align:right;"> Cys </th>
   <th style="text-align:right;"> Gln </th>
   <th style="text-align:right;"> Glu </th>
   <th style="text-align:right;"> Gly </th>
   <th style="text-align:right;"> His </th>
   <th style="text-align:right;"> Ile </th>
   <th style="text-align:right;"> iMet </th>
   <th style="text-align:right;"> Leu </th>
   <th style="text-align:right;"> Lys </th>
   <th style="text-align:right;"> Met </th>
   <th style="text-align:right;"> Phe </th>
   <th style="text-align:right;"> Pro </th>
   <th style="text-align:right;"> Ser </th>
   <th style="text-align:right;"> Thr </th>
   <th style="text-align:right;"> Trp </th>
   <th style="text-align:right;"> Tyr </th>
   <th style="text-align:right;"> Val </th>
  </tr>
 </thead>
<tbody>
  <tr>
   <td style="text-align:left;"> Fungi </td>
   <td style="text-align:right;"> 5 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 89 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> Insecta </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> Mammalia </td>
   <td style="text-align:right;"> 10 </td>
   <td style="text-align:right;"> 4 </td>
   <td style="text-align:right;"> 18 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 9 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 5 </td>
   <td style="text-align:right;"> 4 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 8 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 3 </td>
   <td style="text-align:right;"> 25 </td>
   <td style="text-align:right;"> 17 </td>
   <td style="text-align:right;"> 4 </td>
   <td style="text-align:right;"> 3 </td>
   <td style="text-align:right;"> 5 </td>
   <td style="text-align:right;"> 3 </td>
   <td style="text-align:right;"> 6 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 5 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> Nematoda </td>
   <td style="text-align:right;"> 4 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 3 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 3 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 3 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> Spermatophyta </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 3 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 5 </td>
   <td style="text-align:right;"> 4 </td>
   <td style="text-align:right;"> 3 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 6 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 3 </td>
   <td style="text-align:right;"> 1 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> Vertebrata </td>
   <td style="text-align:right;"> 5 </td>
   <td style="text-align:right;"> 6 </td>
   <td style="text-align:right;"> 18 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 5 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 6 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 5 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 3 </td>
   <td style="text-align:right;"> 9 </td>
   <td style="text-align:right;"> 5 </td>
   <td style="text-align:right;"> 11 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 11 </td>
   <td style="text-align:right;"> 4 </td>
   <td style="text-align:right;"> 11 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 5 </td>
  </tr>
</tbody>
</table>

What's this strange bump in fungal valine tRNAs?

#### Frequency of non-Y11:R24 tRNAs in fungi Val

In [59]:
identities %>% select(clade, quality, isotype, X11.24) %>%
  filter(quality & clade == "Fungi" & isotype == "Val") %>%
  group_by(clade, isotype, X11.24) %>%
  summarize(count = n())

,clade,isotype,X11.24,count
1,Fungi,Val,A:U,1
2,Fungi,Val,C:G,31
3,Fungi,Val,G:C,1
4,Fungi,Val,U:A,194
5,Fungi,Val,U:G,87


There's high frequency of U11:G24 (though technically, it is also a pyrimidine:purine interaction).

#### What's known

- [Ohana et al. 1994](http://dx.doi.org/10.1006/abbi.1994.1503) studied yeast Ser, which contains C11:G24. A switch to G11:C24 decreased aminoacylation efficiency to 78%.
- C11:G24, along with G10:C25, are vital for m$^2_2$G26, according to [Edqvist et al. 1992](https://doi.org/10.1093/nar/20.24.6575). However, other eukaryotes such as _Drosophila_ contain the modification even without these identities.
- U11:A24 prevents RNase P processing in _Drosophila_ ([Levinger et al. 1995](10.1074/jbc.270.32.18903)). It's noted that the tRNA probably folds correctly, and the recognition is stalled by "local helix deformation". 
- This identity element is also important in mitochondrial tRNAs ([Takeuchi et al. 2001)](http://www.jbc.org/content/276/23/20064.full)).

#### What's new

#### Covariance of 10:25 and 11:24 for fungi vs all other clades

In [61]:
identities %>% select(clade, isotype, X10.25, X11.24) %>%
  filter(isotype == "Val") %>%
  mutate(clade=ifelse(clade == "Fungi", "Fungi", "Other")) %>%
  filter(X11.24 %in% c("U:A", "C:G", "U:G")) %>%
  group_by(clade) %>%
  mutate(total=n()) %>%
  group_by(clade, total, X10.25, X11.24) %>%
  summarize(count=n()) %>%
  mutate(freq=count/total) %>%
  filter(freq > 0.01) %>%
  select(-total, -count) %>%
  spread(X10.25, freq, fill=0)

Adding missing grouping variables: `total`


,total,clade,X11.24,G:C,G:U
1,318,Fungi,C:G,0.0880503144654088,0.0283018867924528
2,318,Fungi,U:A,0.553459119496855,0.0534591194968553
3,318,Fungi,U:G,0.273584905660377,0
4,1330,Other,C:G,0.0270676691729323,0
5,1330,Other,U:A,0.927067669172932,0.0360902255639098


A whopping **55%** of fungal valine tRNAs contain G10:C25 and U11:G24. This is very different from other eukaryotic valines, which hovers around 93% for G10:C25 and U11:A24. This seems to be tolerated specifically in fungal valine tRNAs. This could potentially be a switch for modification or for cleavage. Is this enriched in specific species?

In [68]:
identities %>% select(species, clade, isotype, X10.25, X11.24) %>%
  filter(isotype == "Val", clade == "Fungi", X11.24 %in% c("U:A", "C:G", "U:G")) %>%
  mutate(identity = paste0(X10.25, ' / ', X11.24)) %>%
  group_by(species, identity) %>%
  summarize(count = n()) %>%
  spread(identity, count, fill = 0) %>% 
  kable() %>% paste(collapse = '\n') %>% display_markdown

<table>
 <thead>
  <tr>
   <th style="text-align:left;"> species </th>
   <th style="text-align:right;"> G:C / C:G </th>
   <th style="text-align:right;"> G:C / U:A </th>
   <th style="text-align:right;"> G:C / U:G </th>
   <th style="text-align:right;"> G:U / C:G </th>
   <th style="text-align:right;"> G:U / U:A </th>
   <th style="text-align:right;"> U:U / U:A </th>
  </tr>
 </thead>
<tbody>
  <tr>
   <td style="text-align:left;"> ashbGoss_ATCC10895 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 5 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> aspeFumi_AF293 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 3 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> aspeNidu_FGSC_A4 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 3 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> aspeOryz_RIB40 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 3 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> botrCine_B05_10 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> candAlbi_WO_1 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 3 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> candDubl_CD36 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 3 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> candGlab_CBS_138 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 3 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> candOrth_CO_90_125 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> chaeTher_VAR_THERMOPHILUM_DSM1 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 10 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> crypGatt_WM276 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> crypNeof_VAR_GRUBII_H99 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> crypNeof_VAR_NEOFORMANS_B_3501 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> crypNeof_VAR_NEOFORMANS_JEC21 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> cybeJadi_NBRC_0988_NBRC0988 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> debaHans_CBS767 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> enceCuni_GB_M1 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 3 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> enceHell_ATCC50504 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 3 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> enceInte_ATCC50506 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 3 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> enceRoma_SJ_2008 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 3 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> eremCymb_DBVPG#7215 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 3 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> flamVelu_KACC42780 </td>
   <td style="text-align:right;"> 3 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 3 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> fusaGram_CS3005 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 3 </td>
   <td style="text-align:right;"> 5 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> fusaGram_PH_1_NRRL_31084 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 3 </td>
   <td style="text-align:right;"> 7 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> fusaPseu_CS3096 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 3 </td>
   <td style="text-align:right;"> 5 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> fusaPseu_CS3270 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 3 </td>
   <td style="text-align:right;"> 5 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> fusaVert_7600 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 3 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> kazaAfri_CBS_2517 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 3 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> kazaNaga_CBS_8797 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 3 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> kluyLact_NRRL_Y_1140 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 3 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> komaPast_CBS_7435 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> komaPast_GS115 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> lachKluy_NRRL_Y_12651 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> lachTher_CBS_6340 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> magnOryz_70_15 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 6 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> millFari_CBS_7064 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 3 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> myceTher_ATCC42464 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 5 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> naumCast_CBS_4309 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 3 </td>
   <td style="text-align:right;"> 3 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> naumDair_CBS_421 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 4 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> neurCras_OR74A </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 4 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> ogatPara_DL_1 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> peniChry_P2NIAD18 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> sacCer3 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> saccSp_ASHBYA_ACERI </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 4 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> saccSp_BOULARDII_17 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> saccSp_BOULARDII_ATCCMYA_796 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> scheStip_CBS_6054 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 4 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> schiPomb_972H </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 3 </td>
   <td style="text-align:right;"> 1 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> sporReil_SRZ2 </td>
   <td style="text-align:right;"> 5 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> tetrBlat_CBS_6284 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 3 </td>
   <td style="text-align:right;"> 3 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> tetrPhaf_CBS_4417 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> thieTerr_NRRL_8126 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 5 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> toruDelb_CBS_1146 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> ustiMayd_521 </td>
   <td style="text-align:right;"> 3 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> valsMali_03_8 </td>
   <td style="text-align:right;"> 1 </td>
   <td style="text-align:right;"> 9 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> vertDahl_JR2 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 10 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> vertDahl_VDLS_17 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 10 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> yarrLipo_CLIB122 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 5 </td>
   <td style="text-align:right;"> 3 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> yarrLipo_PO1F </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 5 </td>
   <td style="text-align:right;"> 3 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> yarrLipo_WSH_Z06 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 5 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
  <tr>
   <td style="text-align:left;"> zygoRoux_CBS_732 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 2 </td>
   <td style="text-align:right;"> 3 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
  </tr>
</tbody>
</table>

G10:C25 / U11:A24 (canonical) and G10:C25 / U11:G24 (fungal valine) are broadly distributed. Seems like in fungi, both are tolerated by the synthetase or do some other conserved function.

There are a few species that are noteworthy, though:

_Flammulina velutipes_ shows up here again. _Cryptococcus_, _Penicillium chrysogenum_, _Ustilago maydis_, _Sporisorium reilianum_, and _S. pombe_ are all exceptions that contain neither the canonical set or the fungal valine set of features. Instead, they contain a variety of different features.

<img src='figures/fungi-tree.png' style="width: 700px;">

All of the aforementioned species group together. 

### R15:Y48

#### Number of tRNAs without R15:Y48

In [69]:
identities %>% select(clade, species, X15.48) %>%
  filter(!(X15.48 %in% c('G:C', 'A:U'))) %>% nrow()

[1] 1776

#### Which isotypes/clades/species fail the consensus checks?

In [70]:
## Clade/isotype check
best_freqs %>% filter(positions == 'X15.48') %>% group_by(clade, isotype) %>%
  summarize(status=sum((variable %in% c('GC', 'AU', 'PurinePyrimidine'))*count)/sum(count) == 1) %>%
  filter(!status)

## Species check
identities %>% select(clade, species, isotype, X15.48) %>%
  group_by(clade, species, isotype) %>%
  summarize(status=sum((X15.48 %in% c('G:C', 'A:U'))/n()) >= 0.1,
            freq=sum(X15.48 %in% c('G:C', 'A:U')), tRNAs=n()) %>%
  filter(!status)

,clade,isotype,status


,clade,species,isotype,status,freq,tRNAs
1,Fungi,ashbGoss_ATCC10895,Glu,FALSE,0,2
2,Fungi,aspeFumi_AF293,Asp,FALSE,0,2
3,Fungi,aspeNidu_FGSC_A4,Asp,FALSE,0,1
4,Fungi,botrCine_B05_10,Phe,FALSE,0,4
5,Fungi,candGlab_CBS_138,Glu,FALSE,0,2
6,Fungi,candOrth_CO_90_125,Phe,FALSE,0,1
7,Fungi,crypGatt_WM276,Ala,FALSE,0,1
8,Fungi,crypGatt_WM276,Met,FALSE,0,3
9,Fungi,crypNeof_VAR_GRUBII_H99,Ala,FALSE,0,1
10,Fungi,crypNeof_VAR_GRUBII_H99,Gln,FALSE,0,1


This one's the Levitt base pair, and has been shown in E. coli to tolerate different combinations other than R15:Y48. Despite the historical hubbub, it looks like a typical clade/isotype-specific IDE.

### A21 and 8-14-21

#### Number of tRNAs without A21 or U8-A14-A21

In [71]:
identities %>% select(clade, species, X8.14, X21) %>%
  filter(X8.14 != 'U:A' & X21 != 'A') %>% nrow()

identities %>% select(clade, species, X8.14, X21) %>%
  filter(X21 != 'A') %>% nrow()

[1] 5

[1] 236

Even though U:A:A is almost universally conserved, I'm going to focus on A21. The 4 tRNAs without U:A:A can easily be explained away by compensatory tRNAs with U:A:A.

#### Which isotypes/clades/species fail the consensus checks?

In [72]:
## Clade/isotype check
best_freqs %>% filter(positions == 'X21') %>% group_by(clade, isotype) %>%
  summarize(status=sum((variable == 'A')*count)/sum(count) == 1) %>%
  filter(!status)

## Species check
identities %>% select(clade, species, isotype, X21) %>%
  group_by(clade, species, isotype) %>%
  summarize(status=sum(X21 == 'A')/n() >= 0.1,
            freq=sum(X21 == 'A'), tRNAs=n()) %>%
  filter(!status)

,clade,isotype,status


,clade,species,isotype,status,freq,tRNAs
1,Fungi,aspeNidu_FGSC_A4,Phe,0,0,2
2,Fungi,schiPomb_972H,Met,0,0,3
3,Fungi,sporReil_SRZ2,Phe,0,0,4
4,Fungi,ustiMayd_521,Phe,0,0,2


In [77]:
identities %>% select(clade, species, isotype, anticodon, score, isoscore, X8, X14, X15, X16, X17, X17a, X18, X19, X20, X20a, X20b, X21, X22) %>%
  filter((species %in% c("aspeNidu_FGSC_A4", "sporReil_SRZ2", "ustiMayd_521") & isotype == "Phe") | (species %in% c("schiPomb_972H") & isotype == "Met"))

,clade,species,isotype,anticodon,score,isoscore,X8,X14,X15,X16,X17,X17a,X18,X19,X20,X20a,X20b,X21,X22
1,Fungi,aspeNidu_FGSC_A4,Phe,GAA,65,86.7,U,A,G,U,U,-,G,G,U,A,-,U,G
2,Fungi,aspeNidu_FGSC_A4,Phe,GAA,65.1,86.7,U,A,G,U,U,-,G,G,U,A,-,U,G
3,Fungi,schiPomb_972H,Met,CAT,57.1,76.3,U,A,G,U,C,-,G,G,U,-,-,G,G
4,Fungi,schiPomb_972H,Met,CAT,57.2,76.3,U,A,G,U,C,-,G,G,U,-,-,G,G
5,Fungi,schiPomb_972H,Met,CAT,57.2,76.3,U,A,G,U,C,-,G,G,U,-,-,G,G
6,Fungi,sporReil_SRZ2,Phe,GAA,56.5,97.8,U,A,G,U,U,-,G,G,U,A,-,C,G
7,Fungi,sporReil_SRZ2,Phe,GAA,63.8,100,U,A,G,U,U,-,G,G,U,A,-,C,G
8,Fungi,sporReil_SRZ2,Phe,GAA,60.9,101,U,A,G,U,U,-,G,G,U,A,-,C,G
9,Fungi,sporReil_SRZ2,Phe,GAA,60.6,101,U,A,G,U,U,-,G,G,U,A,-,C,G
10,Fungi,ustiMayd_521,Phe,GAA,62.7,93.1,U,A,G,U,U,-,G,G,U,A,-,C,G


It's hard to say whether these are misalignments or not. _S. pombe_ was previously noted as an exception to the A21 rule. We introduce 3 new exceptions with no compensatory tRNAs. It's unusual that all of them are Phe. It's probably not evolutionarily related since there are several more species more related to each of them (e.g., _Aspergillus oryzae_). One possibility is that the fungi Phe synthetase or processing machinery is more permissive in general, and that it can tolerate mutations.

#### What's known

M&G: Always A21, but there are 3 copies of G21 in _S. pombe_. It interacts as U8-A14-A21.

### U33

#### Number of tRNAs without U33

In [78]:
identities %>% select(clade, species, X33) %>%
  filter(X33 != 'U') %>% nrow()

[1] 465

#### Which isotypes/clades/species fail the consensus checks?

In [80]:
## Clade/isotype check
best_freqs %>% filter(positions == 'X33') %>% group_by(clade, isotype) %>%
  summarize(status=sum((variable == 'U')*count)/sum(count) == 1) %>%
  filter(!status)

,clade,isotype,status
1,Insecta,iMet,FALSE
2,Mammalia,iMet,FALSE
3,Nematoda,iMet,FALSE
4,Spermatophyta,iMet,FALSE
5,Vertebrata,iMet,FALSE


This is a pretty strong non-Fungi iMet identity element (which is known). M&G note some Cs here and there, plus a G33 in _Candida albicans_ that disrupts the anticodon arm and deforms the tRNA, decreasing charging efficiency.

## Other thoughts

- Although the synthetase and tRNA should represent a stronger evolutionary signal w.r.t speciation, this isn't the case for tRNA processing that is not isotype-specific.
    - For instance, yeast has an unusually high amount of introns. 
    - This could apply to other processing enzymes, e.g., RNase P, RNase Z, etc.


# Strategy 2: Isotype-specific features

In [86]:
iso_cutoff_freqs = data.frame()
for (cutoff in c(0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99, 1.0)) {
  df = clade_iso_ac_freqs %>%
    group_by(isotype, positions, variable) %>%
    summarize(count=sum(value), freq=sum(value)/sum(total)) %>%
    filter(freq >= cutoff) %>%
    mutate(cutoff=as.character(cutoff)) %>%
    select(isotype, positions, variable, freq, cutoff) %>%
    group_by(isotype, positions) %>%
    arrange(code_groups[variable], desc(freq)) %>%
    filter(row_number(positions) == 1)
  if (nrow(iso_cutoff_freqs) == 0) iso_cutoff_freqs = df
  else iso_cutoff_freqs = rbind(iso_cutoff_freqs, df)
}

In [99]:
iso_cutoff_freqs %>% 
  mutate(variable = as.character(variable)) %>%
  bind_rows(isotype_specific %>% mutate(variable = identity, freq = 0.9, cutoff = "Isotype-specific") %>% select(-identity)) %>%
  filter(positions %in% c(names(paired_positions), names(single_positions))) %>%
  filter(!(positions %in% cutoff_freqs$positions[cutoff_freqs$cutoff == 0.9 & cutoff_freqs$variable %in% c("A", "C", "G", "U", "AU", "UA", "GC", "CG", "UG", "GU")])) %>%
  filter(cutoff %in% c(0.95, 'Isotype-specific')) %>%
  select(-freq) %>%
  spread(cutoff, variable) %>% # generate NAs for missing identities
  gather(cutoff, variable, -isotype, -positions) %>%
  group_by(isotype, positions) %>%
  arrange(cutoff) %>% 
  summarize(identity = paste(variable, collapse = " / ")) %>%
  spread(isotype, identity, fill = '-') %>%
  kable %>% as.character %>% display_markdown

<table>
 <thead>
  <tr>
   <th style="text-align:left;"> positions </th>
   <th style="text-align:left;"> Ala </th>
   <th style="text-align:left;"> Arg </th>
   <th style="text-align:left;"> Asn </th>
   <th style="text-align:left;"> Asp </th>
   <th style="text-align:left;"> Cys </th>
   <th style="text-align:left;"> Gln </th>
   <th style="text-align:left;"> Glu </th>
   <th style="text-align:left;"> Gly </th>
   <th style="text-align:left;"> His </th>
   <th style="text-align:left;"> Ile </th>
   <th style="text-align:left;"> iMet </th>
   <th style="text-align:left;"> Leu </th>
   <th style="text-align:left;"> Lys </th>
   <th style="text-align:left;"> Met </th>
   <th style="text-align:left;"> Phe </th>
   <th style="text-align:left;"> Pro </th>
   <th style="text-align:left;"> Ser </th>
   <th style="text-align:left;"> Thr </th>
   <th style="text-align:left;"> Trp </th>
   <th style="text-align:left;"> Tyr </th>
   <th style="text-align:left;"> Val </th>
  </tr>
 </thead>
<tbody>
  <tr>
   <td style="text-align:left;"> X1.72 </td>
   <td style="text-align:left;"> Paired / GC </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / KetoAmino </td>
   <td style="text-align:left;"> Paired / KetoAmino </td>
   <td style="text-align:left;"> PurinePyrimidine / GC </td>
   <td style="text-align:left;"> GC / GC </td>
   <td style="text-align:left;"> UA / UA </td>
   <td style="text-align:left;"> GC / PurinePyrimidine </td>
   <td style="text-align:left;"> GC / Paired </td>
   <td style="text-align:left;"> GC / GC </td>
   <td style="text-align:left;"> AU / AU </td>
   <td style="text-align:left;"> PurinePyrimidine / GC </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> GC / GC </td>
   <td style="text-align:left;"> StrongPair / GC </td>
   <td style="text-align:left;"> GC / GC </td>
   <td style="text-align:left;"> NA / Paired </td>
   <td style="text-align:left;"> GC / GC </td>
   <td style="text-align:left;"> PurinePyrimidine / GC </td>
   <td style="text-align:left;"> CG / StrongPair </td>
   <td style="text-align:left;"> GC / PurinePyrimidine </td>
  </tr>
  <tr>
   <td style="text-align:left;"> X10.25 </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> GC / GC </td>
   <td style="text-align:left;"> GC / GC </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> GC / Paired </td>
   <td style="text-align:left;"> GC / PurinePyrimidine </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / GU </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> PurinePyrimidine / GC </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> StrongPair / PurinePyrimidine </td>
   <td style="text-align:left;"> GC / GC </td>
   <td style="text-align:left;"> GC / Paired </td>
   <td style="text-align:left;"> Paired / PurinePyrimidine </td>
   <td style="text-align:left;"> GU / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> PurinePyrimidine / GC </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> GC / GC </td>
   <td style="text-align:left;"> Paired / Paired </td>
  </tr>
  <tr>
   <td style="text-align:left;"> X11.24 </td>
   <td style="text-align:left;"> PyrimidinePurine / PyrimidinePurine </td>
   <td style="text-align:left;"> CG / CG </td>
   <td style="text-align:left;"> PyrimidinePurine / CG </td>
   <td style="text-align:left;"> PyrimidinePurine / UA </td>
   <td style="text-align:left;"> CG / PyrimidinePurine </td>
   <td style="text-align:left;"> UA / UA </td>
   <td style="text-align:left;"> UA / UA </td>
   <td style="text-align:left;"> UA / UA </td>
   <td style="text-align:left;"> UA / PyrimidinePurine </td>
   <td style="text-align:left;"> CG / CG </td>
   <td style="text-align:left;"> CG / PyrimidinePurine </td>
   <td style="text-align:left;"> CG / CG </td>
   <td style="text-align:left;"> CG / CG </td>
   <td style="text-align:left;"> PyrimidinePurine / PyrimidinePurine </td>
   <td style="text-align:left;"> CG / CG </td>
   <td style="text-align:left;"> UA / PyrimidinePurine </td>
   <td style="text-align:left;"> CG / CG </td>
   <td style="text-align:left;"> CG / PyrimidinePurine </td>
   <td style="text-align:left;"> PyrimidinePurine / PyrimidinePurine </td>
   <td style="text-align:left;"> PyrimidinePurine / Paired </td>
   <td style="text-align:left;"> Paired / PyrimidinePurine </td>
  </tr>
  <tr>
   <td style="text-align:left;"> X12.23 </td>
   <td style="text-align:left;"> PyrimidinePurine / KetoAmino </td>
   <td style="text-align:left;"> Paired / StrongPair </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> PyrimidinePurine / NA </td>
   <td style="text-align:left;"> StrongPair / Paired </td>
   <td style="text-align:left;"> StrongPair / Paired </td>
   <td style="text-align:left;"> NA / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / KetoAmino </td>
   <td style="text-align:left;"> GC / KetoAmino </td>
   <td style="text-align:left;"> CG / CG </td>
   <td style="text-align:left;"> PyrimidinePurine / UA </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> UA / KetoAmino </td>
   <td style="text-align:left;"> CG / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> PyrimidinePurine / UA </td>
   <td style="text-align:left;"> Paired / NA </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> StrongPair / Paired </td>
  </tr>
  <tr>
   <td style="text-align:left;"> X13.22 </td>
   <td style="text-align:left;"> Paired / NA </td>
   <td style="text-align:left;"> NA / PyrimidinePurine </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> Paired / NA </td>
   <td style="text-align:left;"> CG / Paired </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> CG / CG </td>
   <td style="text-align:left;"> Mismatched / NA </td>
   <td style="text-align:left;"> CG / CG </td>
   <td style="text-align:left;"> Paired / NA </td>
   <td style="text-align:left;"> PyrimidinePurine / CG </td>
   <td style="text-align:left;"> Mismatched / NA </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> CG / Paired </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> - </td>
  </tr>
  <tr>
   <td style="text-align:left;"> X15 </td>
   <td style="text-align:left;"> Purine / V </td>
   <td style="text-align:left;"> Purine / A </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> G / Keto </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> G / Purine </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> G / G </td>
   <td style="text-align:left;"> G / G </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> G / Purine </td>
   <td style="text-align:left;"> G / Strong </td>
   <td style="text-align:left;"> G / G </td>
   <td style="text-align:left;"> G / G </td>
   <td style="text-align:left;"> G / G </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> G / G </td>
   <td style="text-align:left;"> G / G </td>
  </tr>
  <tr>
   <td style="text-align:left;"> X15.48 </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> PurinePyrimidine / AU </td>
   <td style="text-align:left;"> PurinePyrimidine / NA </td>
   <td style="text-align:left;"> PurinePyrimidine / NA </td>
   <td style="text-align:left;"> GC / KetoAmino </td>
   <td style="text-align:left;"> PurinePyrimidine / NA </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> GC / Paired </td>
   <td style="text-align:left;"> PurinePyrimidine / PurinePyrimidine </td>
   <td style="text-align:left;"> GC / GC </td>
   <td style="text-align:left;"> GC / GC </td>
   <td style="text-align:left;"> Paired / PurinePyrimidine </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / NA </td>
   <td style="text-align:left;"> GC / GC </td>
   <td style="text-align:left;"> GC / GC </td>
   <td style="text-align:left;"> GC / GC </td>
   <td style="text-align:left;"> PurinePyrimidine / PurinePyrimidine </td>
   <td style="text-align:left;"> GC / GC </td>
   <td style="text-align:left;"> GC / GC </td>
  </tr>
  <tr>
   <td style="text-align:left;"> X16 </td>
   <td style="text-align:left;"> D / H </td>
   <td style="text-align:left;"> Pyrimidine / U </td>
   <td style="text-align:left;"> Pyrimidine / Pyrimidine </td>
   <td style="text-align:left;"> U / Pyrimidine </td>
   <td style="text-align:left;"> B / NA </td>
   <td style="text-align:left;"> Pyrimidine / Pyrimidine </td>
   <td style="text-align:left;"> Pyrimidine / B </td>
   <td style="text-align:left;"> Pyrimidine / Pyrimidine </td>
   <td style="text-align:left;"> U / Keto </td>
   <td style="text-align:left;"> Pyrimidine / Pyrimidine </td>
   <td style="text-align:left;"> H / NA </td>
   <td style="text-align:left;"> Pyrimidine / Pyrimidine </td>
   <td style="text-align:left;"> Pyrimidine / U </td>
   <td style="text-align:left;"> Pyrimidine / Pyrimidine </td>
   <td style="text-align:left;"> Pyrimidine / H </td>
   <td style="text-align:left;"> D / Keto </td>
   <td style="text-align:left;"> Keto / U </td>
   <td style="text-align:left;"> B / U </td>
   <td style="text-align:left;"> Pyrimidine / Pyrimidine </td>
   <td style="text-align:left;"> Pyrimidine / Pyrimidine </td>
   <td style="text-align:left;"> Pyrimidine / U </td>
  </tr>
  <tr>
   <td style="text-align:left;"> X17 </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> Absent / Absent </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> Absent / Absent </td>
   <td style="text-align:left;"> Absent / NA </td>
   <td style="text-align:left;"> Absent / Absent </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> Absent / Absent </td>
   <td style="text-align:left;"> Absent / Absent </td>
   <td style="text-align:left;"> NA / Pyrimidine </td>
   <td style="text-align:left;"> Absent / Absent </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> NA / H </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> Pyrimidine / NA </td>
   <td style="text-align:left;"> Absent / Absent </td>
   <td style="text-align:left;"> Absent / Absent </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> U / NA </td>
   <td style="text-align:left;"> - </td>
  </tr>
  <tr>
   <td style="text-align:left;"> X17a </td>
   <td style="text-align:left;"> Absent / Absent </td>
   <td style="text-align:left;"> Absent / Absent </td>
   <td style="text-align:left;"> Absent / Absent </td>
   <td style="text-align:left;"> Absent / Absent </td>
   <td style="text-align:left;"> Absent / NA </td>
   <td style="text-align:left;"> Absent / Absent </td>
   <td style="text-align:left;"> Absent / Absent </td>
   <td style="text-align:left;"> Absent / Absent </td>
   <td style="text-align:left;"> Absent / Absent </td>
   <td style="text-align:left;"> Absent / Absent </td>
   <td style="text-align:left;"> Absent / Absent </td>
   <td style="text-align:left;"> Absent / Absent </td>
   <td style="text-align:left;"> Absent / Absent </td>
   <td style="text-align:left;"> Absent / Absent </td>
   <td style="text-align:left;"> Absent / Absent </td>
   <td style="text-align:left;"> Absent / Absent </td>
   <td style="text-align:left;"> Absent / Absent </td>
   <td style="text-align:left;"> Absent / Absent </td>
   <td style="text-align:left;"> Absent / Absent </td>
   <td style="text-align:left;"> Absent / Absent </td>
   <td style="text-align:left;"> Absent / Absent </td>
  </tr>
  <tr>
   <td style="text-align:left;"> X2.71 </td>
   <td style="text-align:left;"> PurinePyrimidine / GC </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> StrongPair / Paired </td>
   <td style="text-align:left;"> GC / PurinePyrimidine </td>
   <td style="text-align:left;"> CG / CG </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> StrongPair / Paired </td>
   <td style="text-align:left;"> Paired / KetoAmino </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> StrongPair / CG </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / StrongPair </td>
   <td style="text-align:left;"> GC / StrongPair </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> StrongPair / StrongPair </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> PyrimidinePurine / PyrimidinePurine </td>
   <td style="text-align:left;"> Paired / Paired </td>
  </tr>
  <tr>
   <td style="text-align:left;"> X20 </td>
   <td style="text-align:left;"> U / U </td>
   <td style="text-align:left;"> H / Weak </td>
   <td style="text-align:left;"> B / Pyrimidine </td>
   <td style="text-align:left;"> Weak / H </td>
   <td style="text-align:left;"> Pyrimidine / Pyrimidine </td>
   <td style="text-align:left;"> U / Pyrimidine </td>
   <td style="text-align:left;"> Pyrimidine / Pyrimidine </td>
   <td style="text-align:left;"> U / U </td>
   <td style="text-align:left;"> U / Pyrimidine </td>
   <td style="text-align:left;"> U / Weak </td>
   <td style="text-align:left;"> Purine / Amino </td>
   <td style="text-align:left;"> U / U </td>
   <td style="text-align:left;"> Pyrimidine / H </td>
   <td style="text-align:left;"> NA / H </td>
   <td style="text-align:left;"> Keto / D </td>
   <td style="text-align:left;"> U / Keto </td>
   <td style="text-align:left;"> U / U </td>
   <td style="text-align:left;"> Pyrimidine / Pyrimidine </td>
   <td style="text-align:left;"> U / Pyrimidine </td>
   <td style="text-align:left;"> Weak / H </td>
   <td style="text-align:left;"> U / U </td>
  </tr>
  <tr>
   <td style="text-align:left;"> X20a </td>
   <td style="text-align:left;"> Absent / NA </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> Absent / NA </td>
   <td style="text-align:left;"> B / NA </td>
   <td style="text-align:left;"> Pyrimidine / U </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> Pyrimidine / Pyrimidine </td>
   <td style="text-align:left;"> Absent / Absent </td>
   <td style="text-align:left;"> Pyrimidine / Pyrimidine </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> Absent / NA </td>
   <td style="text-align:left;"> Absent / Absent </td>
   <td style="text-align:left;"> NA / U </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> Absent / Absent </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> Pyrimidine / U </td>
  </tr>
  <tr>
   <td style="text-align:left;"> X20b </td>
   <td style="text-align:left;"> Absent / Absent </td>
   <td style="text-align:left;"> NA / Absent </td>
   <td style="text-align:left;"> Absent / NA </td>
   <td style="text-align:left;"> Absent / NA </td>
   <td style="text-align:left;"> Absent / Absent </td>
   <td style="text-align:left;"> Absent / Absent </td>
   <td style="text-align:left;"> Absent / Absent </td>
   <td style="text-align:left;"> Absent / NA </td>
   <td style="text-align:left;"> Absent / Absent </td>
   <td style="text-align:left;"> Absent / NA </td>
   <td style="text-align:left;"> Absent / Absent </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> Absent / NA </td>
   <td style="text-align:left;"> Absent / Absent </td>
   <td style="text-align:left;"> Absent / Absent </td>
   <td style="text-align:left;"> Absent / Absent </td>
   <td style="text-align:left;"> Absent / Absent </td>
   <td style="text-align:left;"> Absent / Absent </td>
   <td style="text-align:left;"> Absent / Absent </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> Absent / Absent </td>
  </tr>
  <tr>
   <td style="text-align:left;"> X26 </td>
   <td style="text-align:left;"> Purine / G </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> H / D </td>
   <td style="text-align:left;"> Weak / NA </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> G / G </td>
   <td style="text-align:left;"> G / G </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> Weak / D </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> D / D </td>
   <td style="text-align:left;"> G / Purine </td>
   <td style="text-align:left;"> Purine / Keto </td>
  </tr>
  <tr>
   <td style="text-align:left;"> X27.43 </td>
   <td style="text-align:left;"> Paired / PyrimidinePurine </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / NA </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / NA </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / NA </td>
   <td style="text-align:left;"> PyrimidinePurine / UA </td>
   <td style="text-align:left;"> StrongPair / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> PyrimidinePurine / Paired </td>
   <td style="text-align:left;"> Paired / NA </td>
   <td style="text-align:left;"> Paired / NA </td>
   <td style="text-align:left;"> Paired / CG </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> PyrimidinePurine / PyrimidinePurine </td>
   <td style="text-align:left;"> UA / WeakPair </td>
   <td style="text-align:left;"> Paired / NA </td>
   <td style="text-align:left;"> PyrimidinePurine / Paired </td>
  </tr>
  <tr>
   <td style="text-align:left;"> X28.42 </td>
   <td style="text-align:left;"> NA / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / StrongPair </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / PyrimidinePurine </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> PyrimidinePurine / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
  </tr>
  <tr>
   <td style="text-align:left;"> X29.41 </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> Paired / KetoAmino </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> GC / Paired </td>
   <td style="text-align:left;"> GC / PurinePyrimidine </td>
   <td style="text-align:left;"> PurinePyrimidine / PurinePyrimidine </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> PurinePyrimidine / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> PyrimidinePurine / PyrimidinePurine </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
  </tr>
  <tr>
   <td style="text-align:left;"> X3.70 </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / StrongPair </td>
   <td style="text-align:left;"> PyrimidinePurine / PyrimidinePurine </td>
   <td style="text-align:left;"> PyrimidinePurine / PyrimidinePurine </td>
   <td style="text-align:left;"> Paired / NA </td>
   <td style="text-align:left;"> NA / PyrimidinePurine </td>
   <td style="text-align:left;"> CG / PyrimidinePurine </td>
   <td style="text-align:left;"> PurinePyrimidine / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> PyrimidinePurine / PyrimidinePurine </td>
   <td style="text-align:left;"> CG / CG </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> CG / CG </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> StrongPair / StrongPair </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> PyrimidinePurine / CG </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
  </tr>
  <tr>
   <td style="text-align:left;"> X30.40 </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / PurinePyrimidine </td>
   <td style="text-align:left;"> Paired / StrongPair </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> GC / GC </td>
   <td style="text-align:left;"> GC / GC </td>
   <td style="text-align:left;"> Paired / StrongPair </td>
   <td style="text-align:left;"> Paired / StrongPair </td>
   <td style="text-align:left;"> StrongPair / StrongPair </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> GC / GC </td>
   <td style="text-align:left;"> StrongPair / StrongPair </td>
   <td style="text-align:left;"> GC / GC </td>
   <td style="text-align:left;"> GC / StrongPair </td>
   <td style="text-align:left;"> Paired / GC </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> NA / StrongPair </td>
   <td style="text-align:left;"> StrongPair / StrongPair </td>
   <td style="text-align:left;"> GC / GC </td>
   <td style="text-align:left;"> GC / StrongPair </td>
   <td style="text-align:left;"> StrongPair / StrongPair </td>
  </tr>
  <tr>
   <td style="text-align:left;"> X31.39 </td>
   <td style="text-align:left;"> PyrimidinePurine / Paired </td>
   <td style="text-align:left;"> Paired / WeakPair </td>
   <td style="text-align:left;"> PurinePyrimidine / PurinePyrimidine </td>
   <td style="text-align:left;"> Paired / PyrimidinePurine </td>
   <td style="text-align:left;"> AU / PurinePyrimidine </td>
   <td style="text-align:left;"> PurinePyrimidine / NA </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / PyrimidinePurine </td>
   <td style="text-align:left;"> PurinePyrimidine / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> GC / GC </td>
   <td style="text-align:left;"> Paired / NA </td>
   <td style="text-align:left;"> PurinePyrimidine / PurinePyrimidine </td>
   <td style="text-align:left;"> Mismatched / NA </td>
   <td style="text-align:left;"> AU / AU </td>
   <td style="text-align:left;"> StrongPair / Paired </td>
   <td style="text-align:left;"> Paired / AminoKeto </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> WeakPair / NA </td>
   <td style="text-align:left;"> PurinePyrimidine / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
  </tr>
  <tr>
   <td style="text-align:left;"> X32 </td>
   <td style="text-align:left;"> Pyrimidine / Pyrimidine </td>
   <td style="text-align:left;"> Pyrimidine / C </td>
   <td style="text-align:left;"> C / C </td>
   <td style="text-align:left;"> Pyrimidine / Pyrimidine </td>
   <td style="text-align:left;"> Pyrimidine / Pyrimidine </td>
   <td style="text-align:left;"> Pyrimidine / Pyrimidine </td>
   <td style="text-align:left;"> Pyrimidine / Pyrimidine </td>
   <td style="text-align:left;"> Pyrimidine / Pyrimidine </td>
   <td style="text-align:left;"> U / Pyrimidine </td>
   <td style="text-align:left;"> C / C </td>
   <td style="text-align:left;"> C / C </td>
   <td style="text-align:left;"> Pyrimidine / Pyrimidine </td>
   <td style="text-align:left;"> C / C </td>
   <td style="text-align:left;"> C / C </td>
   <td style="text-align:left;"> C / Pyrimidine </td>
   <td style="text-align:left;"> U / U </td>
   <td style="text-align:left;"> Strong / B </td>
   <td style="text-align:left;"> Pyrimidine / Pyrimidine </td>
   <td style="text-align:left;"> C / Pyrimidine </td>
   <td style="text-align:left;"> C / Pyrimidine </td>
   <td style="text-align:left;"> Pyrimidine / Pyrimidine </td>
  </tr>
  <tr>
   <td style="text-align:left;"> X34 </td>
   <td style="text-align:left;"> NA / D </td>
   <td style="text-align:left;"> H / Pyrimidine </td>
   <td style="text-align:left;"> G / G </td>
   <td style="text-align:left;"> G / G </td>
   <td style="text-align:left;"> G / G </td>
   <td style="text-align:left;"> Pyrimidine / Pyrimidine </td>
   <td style="text-align:left;"> Pyrimidine / Pyrimidine </td>
   <td style="text-align:left;"> B / B </td>
   <td style="text-align:left;"> G / G </td>
   <td style="text-align:left;"> Weak / A </td>
   <td style="text-align:left;"> C / C </td>
   <td style="text-align:left;"> H / Pyrimidine </td>
   <td style="text-align:left;"> Pyrimidine / Pyrimidine </td>
   <td style="text-align:left;"> C / C </td>
   <td style="text-align:left;"> G / G </td>
   <td style="text-align:left;"> H / H </td>
   <td style="text-align:left;"> NA / D </td>
   <td style="text-align:left;"> H / H </td>
   <td style="text-align:left;"> C / C </td>
   <td style="text-align:left;"> G / G </td>
   <td style="text-align:left;"> H / H </td>
  </tr>
  <tr>
   <td style="text-align:left;"> X35 </td>
   <td style="text-align:left;"> G / G </td>
   <td style="text-align:left;"> C / C </td>
   <td style="text-align:left;"> U / U </td>
   <td style="text-align:left;"> U / U </td>
   <td style="text-align:left;"> C / C </td>
   <td style="text-align:left;"> U / U </td>
   <td style="text-align:left;"> U / U </td>
   <td style="text-align:left;"> C / C </td>
   <td style="text-align:left;"> U / U </td>
   <td style="text-align:left;"> A / A </td>
   <td style="text-align:left;"> A / A </td>
   <td style="text-align:left;"> A / A </td>
   <td style="text-align:left;"> U / U </td>
   <td style="text-align:left;"> A / A </td>
   <td style="text-align:left;"> A / A </td>
   <td style="text-align:left;"> G / G </td>
   <td style="text-align:left;"> Strong / Strong </td>
   <td style="text-align:left;"> G / G </td>
   <td style="text-align:left;"> C / C </td>
   <td style="text-align:left;"> U / U </td>
   <td style="text-align:left;"> A / A </td>
  </tr>
  <tr>
   <td style="text-align:left;"> X36 </td>
   <td style="text-align:left;"> C / C </td>
   <td style="text-align:left;"> Keto / Keto </td>
   <td style="text-align:left;"> U / U </td>
   <td style="text-align:left;"> C / C </td>
   <td style="text-align:left;"> A / A </td>
   <td style="text-align:left;"> G / G </td>
   <td style="text-align:left;"> C / C </td>
   <td style="text-align:left;"> C / C </td>
   <td style="text-align:left;"> G / G </td>
   <td style="text-align:left;"> U / U </td>
   <td style="text-align:left;"> U / U </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> U / U </td>
   <td style="text-align:left;"> U / U </td>
   <td style="text-align:left;"> A / A </td>
   <td style="text-align:left;"> G / G </td>
   <td style="text-align:left;"> Weak / Weak </td>
   <td style="text-align:left;"> U / U </td>
   <td style="text-align:left;"> A / A </td>
   <td style="text-align:left;"> A / A </td>
   <td style="text-align:left;"> C / C </td>
  </tr>
  <tr>
   <td style="text-align:left;"> X37 </td>
   <td style="text-align:left;"> A / A </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> A / A </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> A / A </td>
   <td style="text-align:left;"> A / Amino </td>
   <td style="text-align:left;"> A / A </td>
   <td style="text-align:left;"> G / Purine </td>
   <td style="text-align:left;"> Purine / A </td>
   <td style="text-align:left;"> A / A </td>
   <td style="text-align:left;"> G / G </td>
   <td style="text-align:left;"> Purine / A </td>
   <td style="text-align:left;"> A / A </td>
   <td style="text-align:left;"> Purine / G </td>
   <td style="text-align:left;"> G / G </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> A / A </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> A / A </td>
  </tr>
  <tr>
   <td style="text-align:left;"> X38 </td>
   <td style="text-align:left;"> Pyrimidine / Weak </td>
   <td style="text-align:left;"> Amino / A </td>
   <td style="text-align:left;"> A / A </td>
   <td style="text-align:left;"> Pyrimidine / C </td>
   <td style="text-align:left;"> A / Weak </td>
   <td style="text-align:left;"> Weak / Weak </td>
   <td style="text-align:left;"> C / Amino </td>
   <td style="text-align:left;"> H / H </td>
   <td style="text-align:left;"> C / Amino </td>
   <td style="text-align:left;"> A / A </td>
   <td style="text-align:left;"> A / A </td>
   <td style="text-align:left;"> H / H </td>
   <td style="text-align:left;"> A / A </td>
   <td style="text-align:left;"> A / A </td>
   <td style="text-align:left;"> A / Weak </td>
   <td style="text-align:left;"> U / Weak </td>
   <td style="text-align:left;"> A / A </td>
   <td style="text-align:left;"> A / A </td>
   <td style="text-align:left;"> A / Weak </td>
   <td style="text-align:left;"> A / Amino </td>
   <td style="text-align:left;"> Pyrimidine / C </td>
  </tr>
  <tr>
   <td style="text-align:left;"> X4.69 </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / CG </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / NA </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / NA </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / PyrimidinePurine </td>
   <td style="text-align:left;"> PurinePyrimidine / Paired </td>
   <td style="text-align:left;"> Paired / PurinePyrimidine </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / NA </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> PyrimidinePurine / PyrimidinePurine </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> NA / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
  </tr>
  <tr>
   <td style="text-align:left;"> X44 </td>
   <td style="text-align:left;"> A / A </td>
   <td style="text-align:left;"> V / A </td>
   <td style="text-align:left;"> Weak / Amino </td>
   <td style="text-align:left;"> D / NA </td>
   <td style="text-align:left;"> Weak / H </td>
   <td style="text-align:left;"> Pyrimidine / H </td>
   <td style="text-align:left;"> H / H </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> H / NA </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> A / A </td>
   <td style="text-align:left;"> U / U </td>
   <td style="text-align:left;"> D / NA </td>
   <td style="text-align:left;"> A / A </td>
   <td style="text-align:left;"> Purine / V </td>
   <td style="text-align:left;"> A / Weak </td>
   <td style="text-align:left;"> Weak / Weak </td>
   <td style="text-align:left;"> Purine / A </td>
   <td style="text-align:left;"> Purine / V </td>
   <td style="text-align:left;"> A / D </td>
   <td style="text-align:left;"> A / A </td>
  </tr>
  <tr>
   <td style="text-align:left;"> X45 </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> G / G </td>
   <td style="text-align:left;"> G / G </td>
   <td style="text-align:left;"> G / G </td>
   <td style="text-align:left;"> G / G </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> Purine / NA </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> Keto / D </td>
   <td style="text-align:left;"> Absent / Absent </td>
   <td style="text-align:left;"> Keto / Keto </td>
   <td style="text-align:left;"> G / NA </td>
   <td style="text-align:left;"> Keto / D </td>
   <td style="text-align:left;"> G / G </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> G / G </td>
   <td style="text-align:left;"> G / Purine </td>
   <td style="text-align:left;"> G / B </td>
   <td style="text-align:left;"> Strong / G </td>
  </tr>
  <tr>
   <td style="text-align:left;"> X46 </td>
   <td style="text-align:left;"> G / G </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> Purine / A </td>
   <td style="text-align:left;"> G / G </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> G / G </td>
   <td style="text-align:left;"> Purine / G </td>
   <td style="text-align:left;"> G / G </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> G / G </td>
   <td style="text-align:left;"> G / G </td>
   <td style="text-align:left;"> G / G </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> G / G </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> Purine / G </td>
  </tr>
  <tr>
   <td style="text-align:left;"> X47 </td>
   <td style="text-align:left;"> Pyrimidine / B </td>
   <td style="text-align:left;"> Pyrimidine / U </td>
   <td style="text-align:left;"> U / B </td>
   <td style="text-align:left;"> NA / Absent </td>
   <td style="text-align:left;"> U / U </td>
   <td style="text-align:left;"> Absent / Absent </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> Absent / NA </td>
   <td style="text-align:left;"> U / Pyrimidine </td>
   <td style="text-align:left;"> U / Weak </td>
   <td style="text-align:left;"> Absent / Absent </td>
   <td style="text-align:left;"> Pyrimidine / Pyrimidine </td>
   <td style="text-align:left;"> Pyrimidine / NA </td>
   <td style="text-align:left;"> U / Pyrimidine </td>
   <td style="text-align:left;"> Pyrimidine / Pyrimidine </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> Pyrimidine / U </td>
   <td style="text-align:left;"> Pyrimidine / Pyrimidine </td>
   <td style="text-align:left;"> U / Pyrimidine </td>
   <td style="text-align:left;"> Pyrimidine / U </td>
  </tr>
  <tr>
   <td style="text-align:left;"> X48 </td>
   <td style="text-align:left;"> Amino / Amino </td>
   <td style="text-align:left;"> Pyrimidine / U </td>
   <td style="text-align:left;"> Pyrimidine / Pyrimidine </td>
   <td style="text-align:left;"> Pyrimidine / Pyrimidine </td>
   <td style="text-align:left;"> C / Amino </td>
   <td style="text-align:left;"> Pyrimidine / Pyrimidine </td>
   <td style="text-align:left;"> C / Pyrimidine </td>
   <td style="text-align:left;"> Pyrimidine / C </td>
   <td style="text-align:left;"> C / Pyrimidine </td>
   <td style="text-align:left;"> Pyrimidine / C </td>
   <td style="text-align:left;"> C / C </td>
   <td style="text-align:left;"> C / C </td>
   <td style="text-align:left;"> Pyrimidine / Pyrimidine </td>
   <td style="text-align:left;"> Pyrimidine / Pyrimidine </td>
   <td style="text-align:left;"> C / Amino </td>
   <td style="text-align:left;"> C / C </td>
   <td style="text-align:left;"> C / C </td>
   <td style="text-align:left;"> C / C </td>
   <td style="text-align:left;"> Pyrimidine / Pyrimidine </td>
   <td style="text-align:left;"> C / C </td>
   <td style="text-align:left;"> C / C </td>
  </tr>
  <tr>
   <td style="text-align:left;"> X49.65 </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> StrongPair / StrongPair </td>
   <td style="text-align:left;"> CG / CG </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> CG / Paired </td>
   <td style="text-align:left;"> StrongPair / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> StrongPair / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / GC </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> StrongPair / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / AminoKeto </td>
   <td style="text-align:left;"> Paired / StrongPair </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> PurinePyrimidine / Paired </td>
   <td style="text-align:left;"> Paired / StrongPair </td>
  </tr>
  <tr>
   <td style="text-align:left;"> X5.68 </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / NA </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / NA </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
  </tr>
  <tr>
   <td style="text-align:left;"> X50.64 </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / PyrimidinePurine </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> PyrimidinePurine / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / StrongPair </td>
   <td style="text-align:left;"> PyrimidinePurine / Paired </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / PyrimidinePurine </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> PyrimidinePurine / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> StrongPair / Paired </td>
   <td style="text-align:left;"> Paired / PyrimidinePurine </td>
  </tr>
  <tr>
   <td style="text-align:left;"> X51.63 </td>
   <td style="text-align:left;"> Paired / StrongPair </td>
   <td style="text-align:left;"> PurinePyrimidine / PurinePyrimidine </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> PurinePyrimidine / GC </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> StrongPair / NA </td>
   <td style="text-align:left;"> Paired / PurinePyrimidine </td>
   <td style="text-align:left;"> Paired / GC </td>
   <td style="text-align:left;"> Paired / NA </td>
   <td style="text-align:left;"> Paired / PurinePyrimidine </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> GC / GC </td>
   <td style="text-align:left;"> GC / GC </td>
   <td style="text-align:left;"> PurinePyrimidine / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> GC / StrongPair </td>
   <td style="text-align:left;"> PurinePyrimidine / PurinePyrimidine </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
  </tr>
  <tr>
   <td style="text-align:left;"> X52.62 </td>
   <td style="text-align:left;"> GC / PurinePyrimidine </td>
   <td style="text-align:left;"> GC / GC </td>
   <td style="text-align:left;"> PurinePyrimidine / PurinePyrimidine </td>
   <td style="text-align:left;"> GC / PurinePyrimidine </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> PurinePyrimidine / PurinePyrimidine </td>
   <td style="text-align:left;"> GC / GC </td>
   <td style="text-align:left;"> PurinePyrimidine / StrongPair </td>
   <td style="text-align:left;"> Paired / NA </td>
   <td style="text-align:left;"> PurinePyrimidine / PurinePyrimidine </td>
   <td style="text-align:left;"> PurinePyrimidine / Paired </td>
   <td style="text-align:left;"> PurinePyrimidine / PurinePyrimidine </td>
   <td style="text-align:left;"> Paired / PurinePyrimidine </td>
   <td style="text-align:left;"> PurinePyrimidine / NA </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> PurinePyrimidine / Paired </td>
   <td style="text-align:left;"> PurinePyrimidine / PurinePyrimidine </td>
   <td style="text-align:left;"> PurinePyrimidine / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> PurinePyrimidine / PurinePyrimidine </td>
  </tr>
  <tr>
   <td style="text-align:left;"> X57 </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> Purine / G </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> Purine / G </td>
   <td style="text-align:left;"> G / G </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> G / G </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> Purine / G </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> Purine / G </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> Purine / G </td>
  </tr>
  <tr>
   <td style="text-align:left;"> X59 </td>
   <td style="text-align:left;"> Weak / H </td>
   <td style="text-align:left;"> NA / B </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> Weak / H </td>
   <td style="text-align:left;"> D / NA </td>
   <td style="text-align:left;"> NA / H </td>
   <td style="text-align:left;"> B / NA </td>
   <td style="text-align:left;"> Pyrimidine / H </td>
   <td style="text-align:left;"> Weak / D </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> A / A </td>
   <td style="text-align:left;"> A / A </td>
   <td style="text-align:left;"> B / Strong </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> Pyrimidine / D </td>
   <td style="text-align:left;"> H / Weak </td>
   <td style="text-align:left;"> Weak / Weak </td>
   <td style="text-align:left;"> H / Weak </td>
   <td style="text-align:left;"> D / D </td>
   <td style="text-align:left;"> H / H </td>
   <td style="text-align:left;"> D / D </td>
  </tr>
  <tr>
   <td style="text-align:left;"> X6.67 </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> PyrimidinePurine / Paired </td>
   <td style="text-align:left;"> Paired / NA </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / NA </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / NA </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
  </tr>
  <tr>
   <td style="text-align:left;"> X60 </td>
   <td style="text-align:left;"> B / H </td>
   <td style="text-align:left;"> Pyrimidine / U </td>
   <td style="text-align:left;"> Pyrimidine / Pyrimidine </td>
   <td style="text-align:left;"> Pyrimidine / Pyrimidine </td>
   <td style="text-align:left;"> Pyrimidine / Pyrimidine </td>
   <td style="text-align:left;"> Pyrimidine / Pyrimidine </td>
   <td style="text-align:left;"> Pyrimidine / U </td>
   <td style="text-align:left;"> U / Pyrimidine </td>
   <td style="text-align:left;"> U / U </td>
   <td style="text-align:left;"> H / H </td>
   <td style="text-align:left;"> A / A </td>
   <td style="text-align:left;"> Pyrimidine / Pyrimidine </td>
   <td style="text-align:left;"> Pyrimidine / Pyrimidine </td>
   <td style="text-align:left;"> Pyrimidine / B </td>
   <td style="text-align:left;"> Pyrimidine / C </td>
   <td style="text-align:left;"> Pyrimidine / Pyrimidine </td>
   <td style="text-align:left;"> U / Pyrimidine </td>
   <td style="text-align:left;"> Pyrimidine / Pyrimidine </td>
   <td style="text-align:left;"> U / Pyrimidine </td>
   <td style="text-align:left;"> Pyrimidine / Pyrimidine </td>
   <td style="text-align:left;"> H / H </td>
  </tr>
  <tr>
   <td style="text-align:left;"> X7.66 </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / StrongPair </td>
   <td style="text-align:left;"> Paired / PurinePyrimidine </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> PurinePyrimidine / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / WeakPair </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
   <td style="text-align:left;"> Paired / Paired </td>
  </tr>
  <tr>
   <td style="text-align:left;"> X73 </td>
   <td style="text-align:left;"> D / Purine </td>
   <td style="text-align:left;"> D / Keto </td>
   <td style="text-align:left;"> Purine / G </td>
   <td style="text-align:left;"> Purine / G </td>
   <td style="text-align:left;"> U / U </td>
   <td style="text-align:left;"> U / Weak </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> A / A </td>
   <td style="text-align:left;"> A / Weak </td>
   <td style="text-align:left;"> A / A </td>
   <td style="text-align:left;"> A / A </td>
   <td style="text-align:left;"> A / A </td>
   <td style="text-align:left;"> D / Keto </td>
   <td style="text-align:left;"> A / A </td>
   <td style="text-align:left;"> A / A </td>
   <td style="text-align:left;"> Pyrimidine / C </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> Weak / Weak </td>
   <td style="text-align:left;"> A / A </td>
   <td style="text-align:left;"> A / A </td>
   <td style="text-align:left;"> Weak / A </td>
  </tr>
  <tr>
   <td style="text-align:left;"> X9 </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> V / D </td>
   <td style="text-align:left;"> Purine / V </td>
   <td style="text-align:left;"> Amino / V </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> G / Purine </td>
   <td style="text-align:left;"> G / G </td>
   <td style="text-align:left;"> Purine / A </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> G / Purine </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> Purine / Purine </td>
   <td style="text-align:left;"> Purine / Purine </td>
  </tr>
</tbody>
</table>

Literally hundreds of candidates here. I want to choose a position that is (a) not involved in 3D interactions, (b) not involved in base pairing, (c) has clear isotype specificity, (d) does not have clear functionality (aside from intron splicing, but this is an exception), (e) is a single position (we looked at a lot of base pairs above), and (f) differs between 99% and isotype-specific cutoffs. It'd also be nice if the isotype is not Pro and is of Type I, and the exception is not from Fungi.

First - we try Tyr-A38:

In [100]:
iso_cutoff_freqs %>% mutate(variable = as.character(variable)) %>%
  bind_rows(isotype_specific %>% mutate(variable = identity, freq = 0.9, cutoff = "Isotype-specific") %>% select(-identity)) %>%
  filter(isotype == "Tyr" & positions == "X38") %>%
  spread(cutoff, variable, fill = '-') %>% select(-freq)

,isotype,positions,0.5,0.6,0.7,0.8,0.9,0.95,0.99,Isotype-specific
1,Tyr,X38,-,-,-,-,-,-,-,Amino
2,Tyr,X38,A,A,A,A,A,A,-,-
3,Tyr,X38,-,-,-,-,-,-,Purine,-


#### No. of Tyr RNAs without A38

In [102]:
identities %>% select(clade, species, isotype, X38) %>%
  filter(isotype == "Tyr" & X38 != 'A') %>% nrow()

[1] 12

#### Which species or clades fail the check?

In [106]:
## Clade check
best_freqs %>% filter(positions == 'X38' & isotype == "Tyr") %>% group_by(clade) %>%
  summarize(status=sum((variable == 'A')*count)/sum(count) == 1) %>%
  filter(!status)

## Species check
identities %>% select(clade, species, isotype, X38) %>%
  filter(isotype == "Tyr") %>%
  group_by(clade, species) %>%
  summarize(status=sum(X38 == 'A')/n() >= 0.1,
            freq=sum(X38 == 'A'), tRNAs=n()) %>%
  filter(!status)

,clade,status


,clade,species,status,freq,tRNAs
1,Fungi,sporReil_SRZ2,0,0,2
2,Fungi,ustiMayd_521,0,0,1


Yet another example with these two fungi. Next, we try Asp-U59:

#### No. of Asp RNAs without U59

In [107]:
iso_cutoff_freqs %>% mutate(variable = as.character(variable)) %>%
  bind_rows(isotype_specific %>% mutate(variable = identity, freq = 0.9, cutoff = "Isotype-specific") %>% select(-identity)) %>%
  filter(isotype == "Asp" & positions == "X59") %>%
  spread(cutoff, variable, fill = '-') %>% select(-freq)

,isotype,positions,0.5,0.6,0.7,0.8,0.9,0.95,Isotype-specific
1,Asp,X59,-,-,-,-,-,-,H
2,Asp,X59,U,U,U,U,U,-,-
3,Asp,X59,-,-,-,-,-,Weak,-


#### Which species or clades fail the check?

In [108]:
## Clade check
best_freqs %>% filter(positions == 'X59' & isotype == "Asp") %>% group_by(clade) %>%
  summarize(status=sum((variable == 'U')*count)/sum(count) == 1) %>%
  filter(!status)

## Species check
identities %>% select(clade, species, isotype, X59) %>%
  filter(isotype == "Asp") %>%
  group_by(clade, species) %>%
  summarize(status=sum(X59 == 'U')/n() >= 0.1,
            freq=sum(X59 == 'U'), tRNAs=n()) %>%
  filter(!status)

,clade,status


,clade,species,status,freq,tRNAs
1,Fungi,aspeFumi_AF293,0,0,2
2,Fungi,aspeNidu_FGSC_A4,0,0,1
3,Fungi,enceCuni_GB_M1,0,0,1
4,Fungi,enceHell_ATCC50504,0,0,1
5,Fungi,enceInte_ATCC50506,0,0,1
6,Fungi,enceRoma_SJ_2008,0,0,1


More fungi examples. Next, we try Ile 29:41.

#### Ile G29:C41

In [110]:
## No. tRNAs without G29:C41
identities %>% select(clade, species, isotype, X29.41) %>%
  filter(isotype == "Ile" & X29.41 != 'G:C') %>% nrow()

## Clade check
best_freqs %>% filter(positions == 'X29.41' & isotype == "Ile") %>% group_by(clade) %>%
  summarize(status=sum((variable == 'GC')*count)/sum(count) == 1) %>%
  filter(!status)

## Species check
identities %>% select(clade, species, isotype, X29.41) %>%
  filter(isotype == "Ile") %>%
  group_by(clade, species) %>%
  summarize(status=sum(X29.41 == 'G:C')/n() >= 0.1,
            freq=sum(X29.41 == 'G:C'), tRNAs=n()) %>%
  filter(!status)

[1] 40

,clade,status


,clade,species,status,freq,tRNAs
1,Fungi,sporReil_SRZ2,0,0,5
2,Fungi,ustiMayd_521,0,0,5


#### Cys A38

In [112]:
## No. tRNAs without A38
identities %>% select(clade, species, isotype, X38) %>%
  filter(isotype == "Cys" & X38 != 'A') %>% nrow()

## Clade check
best_freqs %>% filter(positions == 'X38' & isotype == "Cys") %>% group_by(clade) %>%
  summarize(status=sum((variable == 'A')*count)/sum(count) == 1) %>%
  filter(!status)

## Species check
identities %>% select(clade, species, isotype, X38) %>%
  filter(isotype == "Cys") %>%
  group_by(clade, species) %>%
  summarize(status=sum(X38 == 'A')/n() >= 0.1,
            freq=sum(X38 == 'A'), tRNAs=n()) %>%
  filter(!status)

[1] 30

,clade,status


,clade,species,status,freq,tRNAs
1,Fungi,aspeNidu_FGSC_A4,0,0,1
2,Fungi,chaeTher_VAR_THERMOPHILUM_DSM1,0,0,1
3,Fungi,myceTher_ATCC42464,0,0,2
4,Fungi,neurCras_OR74A,0,0,1
5,Fungi,thieTerr_NRRL_8126,0,0,2
6,Nematoda,panRed1,0,0,1


Finally, an example outside of fungi!

In [114]:
identities %>% filter(species == "panRed1", isotype == "Cys") %>%
  kable %>% as.character %>% display_markdown

<table>
 <thead>
  <tr>
   <th style="text-align:left;"> clade </th>
   <th style="text-align:left;"> domain </th>
   <th style="text-align:left;"> isotype </th>
   <th style="text-align:left;"> seqname </th>
   <th style="text-align:left;"> species </th>
   <th style="text-align:left;"> species_long </th>
   <th style="text-align:left;"> taxid </th>
   <th style="text-align:left;"> isotype_best </th>
   <th style="text-align:left;"> anticodon </th>
   <th style="text-align:right;"> score </th>
   <th style="text-align:right;"> isoscore </th>
   <th style="text-align:right;"> GC </th>
   <th style="text-align:right;"> intron </th>
   <th style="text-align:right;"> insertions </th>
   <th style="text-align:right;"> deletions </th>
   <th style="text-align:right;"> D.loop </th>
   <th style="text-align:right;"> AC.loop </th>
   <th style="text-align:right;"> TPC.loop </th>
   <th style="text-align:right;"> V.arm </th>
   <th style="text-align:left;"> quality </th>
   <th style="text-align:left;"> restrict </th>
   <th style="text-align:left;"> X1.72 </th>
   <th style="text-align:left;"> X1 </th>
   <th style="text-align:left;"> X1i1 </th>
   <th style="text-align:left;"> X2.71 </th>
   <th style="text-align:left;"> X2 </th>
   <th style="text-align:left;"> X2i1 </th>
   <th style="text-align:left;"> X3.70 </th>
   <th style="text-align:left;"> X3 </th>
   <th style="text-align:left;"> X3i1 </th>
   <th style="text-align:left;"> X3i2 </th>
   <th style="text-align:left;"> X3i3 </th>
   <th style="text-align:left;"> X3i4 </th>
   <th style="text-align:left;"> X3i5 </th>
   <th style="text-align:left;"> X3i6 </th>
   <th style="text-align:left;"> X4.69 </th>
   <th style="text-align:left;"> X4 </th>
   <th style="text-align:left;"> X4i1 </th>
   <th style="text-align:left;"> X4i2 </th>
   <th style="text-align:left;"> X4i3 </th>
   <th style="text-align:left;"> X4i4 </th>
   <th style="text-align:left;"> X4i5 </th>
   <th style="text-align:left;"> X4i6 </th>
   <th style="text-align:left;"> X4i7 </th>
   <th style="text-align:left;"> X4i8 </th>
   <th style="text-align:left;"> X4i9 </th>
   <th style="text-align:left;"> X4i10 </th>
   <th style="text-align:left;"> X4i11 </th>
   <th style="text-align:left;"> X5.68 </th>
   <th style="text-align:left;"> X5 </th>
   <th style="text-align:left;"> X5i1 </th>
   <th style="text-align:left;"> X5i2 </th>
   <th style="text-align:left;"> X5i3 </th>
   <th style="text-align:left;"> X5i4 </th>
   <th style="text-align:left;"> X5i5 </th>
   <th style="text-align:left;"> X5i6 </th>
   <th style="text-align:left;"> X6.67 </th>
   <th style="text-align:left;"> X6 </th>
   <th style="text-align:left;"> X6i1 </th>
   <th style="text-align:left;"> X7.66 </th>
   <th style="text-align:left;"> X7 </th>
   <th style="text-align:left;"> X7i1 </th>
   <th style="text-align:left;"> X7i2 </th>
   <th style="text-align:left;"> X7i3 </th>
   <th style="text-align:left;"> X7i4 </th>
   <th style="text-align:left;"> X7i5 </th>
   <th style="text-align:left;"> X7i6 </th>
   <th style="text-align:left;"> X7i7 </th>
   <th style="text-align:left;"> X7i8 </th>
   <th style="text-align:left;"> X7i9 </th>
   <th style="text-align:left;"> X8 </th>
   <th style="text-align:left;"> X8.14.21 </th>
   <th style="text-align:left;"> X8.14 </th>
   <th style="text-align:left;"> X8i1 </th>
   <th style="text-align:left;"> X8i2 </th>
   <th style="text-align:left;"> X9 </th>
   <th style="text-align:left;"> X9.12.23 </th>
   <th style="text-align:left;"> X9.23 </th>
   <th style="text-align:left;"> X9i1 </th>
   <th style="text-align:left;"> X10.25 </th>
   <th style="text-align:left;"> X10 </th>
   <th style="text-align:left;"> X10.25.45 </th>
   <th style="text-align:left;"> X10.45 </th>
   <th style="text-align:left;"> X10i1 </th>
   <th style="text-align:left;"> X11.24 </th>
   <th style="text-align:left;"> X11 </th>
   <th style="text-align:left;"> X12.23 </th>
   <th style="text-align:left;"> X12 </th>
   <th style="text-align:left;"> X12i1 </th>
   <th style="text-align:left;"> X13.22 </th>
   <th style="text-align:left;"> X13 </th>
   <th style="text-align:left;"> X13.22.46 </th>
   <th style="text-align:left;"> X14 </th>
   <th style="text-align:left;"> X14i1 </th>
   <th style="text-align:left;"> X14i2 </th>
   <th style="text-align:left;"> X14i3 </th>
   <th style="text-align:left;"> X14i4 </th>
   <th style="text-align:left;"> X14i5 </th>
   <th style="text-align:left;"> X14i6 </th>
   <th style="text-align:left;"> X14i7 </th>
   <th style="text-align:left;"> X14i8 </th>
   <th style="text-align:left;"> X14i9 </th>
   <th style="text-align:left;"> X14i10 </th>
   <th style="text-align:left;"> X14i11 </th>
   <th style="text-align:left;"> X14i12 </th>
   <th style="text-align:left;"> X14i13 </th>
   <th style="text-align:left;"> X15 </th>
   <th style="text-align:left;"> X15.48 </th>
   <th style="text-align:left;"> X16 </th>
   <th style="text-align:left;"> X16i1 </th>
   <th style="text-align:left;"> X16i2 </th>
   <th style="text-align:left;"> X16i3 </th>
   <th style="text-align:left;"> X16i4 </th>
   <th style="text-align:left;"> X16i5 </th>
   <th style="text-align:left;"> X16i6 </th>
   <th style="text-align:left;"> X16i7 </th>
   <th style="text-align:left;"> X16i8 </th>
   <th style="text-align:left;"> X16i9 </th>
   <th style="text-align:left;"> X16i10 </th>
   <th style="text-align:left;"> X16i11 </th>
   <th style="text-align:left;"> X16i12 </th>
   <th style="text-align:left;"> X16i13 </th>
   <th style="text-align:left;"> X16i14 </th>
   <th style="text-align:left;"> X16i15 </th>
   <th style="text-align:left;"> X16i16 </th>
   <th style="text-align:left;"> X17 </th>
   <th style="text-align:left;"> X17a </th>
   <th style="text-align:left;"> X18 </th>
   <th style="text-align:left;"> X18.55 </th>
   <th style="text-align:left;"> X19 </th>
   <th style="text-align:left;"> X19.56 </th>
   <th style="text-align:left;"> X19i1 </th>
   <th style="text-align:left;"> X19i2 </th>
   <th style="text-align:left;"> X19i3 </th>
   <th style="text-align:left;"> X19i4 </th>
   <th style="text-align:left;"> X19i5 </th>
   <th style="text-align:left;"> X19i6 </th>
   <th style="text-align:left;"> X19i7 </th>
   <th style="text-align:left;"> X19i8 </th>
   <th style="text-align:left;"> X19i9 </th>
   <th style="text-align:left;"> X19i10 </th>
   <th style="text-align:left;"> X20 </th>
   <th style="text-align:left;"> X20i1 </th>
   <th style="text-align:left;"> X20i2 </th>
   <th style="text-align:left;"> X20i3 </th>
   <th style="text-align:left;"> X20i4 </th>
   <th style="text-align:left;"> X20a </th>
   <th style="text-align:left;"> X20b </th>
   <th style="text-align:left;"> X21 </th>
   <th style="text-align:left;"> X22 </th>
   <th style="text-align:left;"> X22.46 </th>
   <th style="text-align:left;"> X22i1 </th>
   <th style="text-align:left;"> X23 </th>
   <th style="text-align:left;"> X23i1 </th>
   <th style="text-align:left;"> X23i2 </th>
   <th style="text-align:left;"> X23i3 </th>
   <th style="text-align:left;"> X23i4 </th>
   <th style="text-align:left;"> X23i5 </th>
   <th style="text-align:left;"> X23i6 </th>
   <th style="text-align:left;"> X24 </th>
   <th style="text-align:left;"> X24i1 </th>
   <th style="text-align:left;"> X25 </th>
   <th style="text-align:left;"> X25i1 </th>
   <th style="text-align:left;"> X25i2 </th>
   <th style="text-align:left;"> X25i3 </th>
   <th style="text-align:left;"> X25i4 </th>
   <th style="text-align:left;"> X26 </th>
   <th style="text-align:left;"> X26.44 </th>
   <th style="text-align:left;"> X26i1 </th>
   <th style="text-align:left;"> X26i2 </th>
   <th style="text-align:left;"> X26i3 </th>
   <th style="text-align:left;"> X26i4 </th>
   <th style="text-align:left;"> X26i5 </th>
   <th style="text-align:left;"> X26i6 </th>
   <th style="text-align:left;"> X26i7 </th>
   <th style="text-align:left;"> X26i8 </th>
   <th style="text-align:left;"> X26i9 </th>
   <th style="text-align:left;"> X26i10 </th>
   <th style="text-align:left;"> X27.43 </th>
   <th style="text-align:left;"> X27 </th>
   <th style="text-align:left;"> X27i1 </th>
   <th style="text-align:left;"> X27i2 </th>
   <th style="text-align:left;"> X27i3 </th>
   <th style="text-align:left;"> X27i4 </th>
   <th style="text-align:left;"> X28.42 </th>
   <th style="text-align:left;"> X28 </th>
   <th style="text-align:left;"> X28i1 </th>
   <th style="text-align:left;"> X29.41 </th>
   <th style="text-align:left;"> X29 </th>
   <th style="text-align:left;"> X29i1 </th>
   <th style="text-align:left;"> X29i2 </th>
   <th style="text-align:left;"> X30.40 </th>
   <th style="text-align:left;"> X30 </th>
   <th style="text-align:left;"> X30i1 </th>
   <th style="text-align:left;"> X31.39 </th>
   <th style="text-align:left;"> X31 </th>
   <th style="text-align:left;"> X32 </th>
   <th style="text-align:left;"> X33 </th>
   <th style="text-align:left;"> X34 </th>
   <th style="text-align:left;"> X35 </th>
   <th style="text-align:left;"> X35i1 </th>
   <th style="text-align:left;"> X36 </th>
   <th style="text-align:left;"> X37 </th>
   <th style="text-align:left;"> X37i1 </th>
   <th style="text-align:left;"> X37i2 </th>
   <th style="text-align:left;"> X37i3 </th>
   <th style="text-align:left;"> X37i4 </th>
   <th style="text-align:left;"> X37i5 </th>
   <th style="text-align:left;"> X37i6 </th>
   <th style="text-align:left;"> X37i7 </th>
   <th style="text-align:left;"> X37i8 </th>
   <th style="text-align:left;"> X37i9 </th>
   <th style="text-align:left;"> X38 </th>
   <th style="text-align:left;"> X38i1 </th>
   <th style="text-align:left;"> X39 </th>
   <th style="text-align:left;"> X39i1 </th>
   <th style="text-align:left;"> X39i2 </th>
   <th style="text-align:left;"> X40 </th>
   <th style="text-align:left;"> X40i1 </th>
   <th style="text-align:left;"> X40i2 </th>
   <th style="text-align:left;"> X41 </th>
   <th style="text-align:left;"> X41i1 </th>
   <th style="text-align:left;"> X41i2 </th>
   <th style="text-align:left;"> X42 </th>
   <th style="text-align:left;"> X42i1 </th>
   <th style="text-align:left;"> X43 </th>
   <th style="text-align:left;"> X44 </th>
   <th style="text-align:left;"> X44i1 </th>
   <th style="text-align:left;"> X44i2 </th>
   <th style="text-align:left;"> X44i3 </th>
   <th style="text-align:left;"> X44i4 </th>
   <th style="text-align:left;"> X44i5 </th>
   <th style="text-align:left;"> X44i6 </th>
   <th style="text-align:left;"> X44i7 </th>
   <th style="text-align:left;"> X44i8 </th>
   <th style="text-align:left;"> X44i9 </th>
   <th style="text-align:left;"> X44i10 </th>
   <th style="text-align:left;"> X44i11 </th>
   <th style="text-align:left;"> X44i12 </th>
   <th style="text-align:left;"> X44i13 </th>
   <th style="text-align:left;"> X44i14 </th>
   <th style="text-align:left;"> X44i15 </th>
   <th style="text-align:left;"> X44i16 </th>
   <th style="text-align:left;"> X44i17 </th>
   <th style="text-align:left;"> X44i18 </th>
   <th style="text-align:left;"> X44i19 </th>
   <th style="text-align:left;"> X44i20 </th>
   <th style="text-align:left;"> X44i21 </th>
   <th style="text-align:left;"> X44i22 </th>
   <th style="text-align:left;"> X44i23 </th>
   <th style="text-align:left;"> X45 </th>
   <th style="text-align:left;"> V11.V21 </th>
   <th style="text-align:left;"> V12.V22 </th>
   <th style="text-align:left;"> V13.V23 </th>
   <th style="text-align:left;"> V14.V24 </th>
   <th style="text-align:left;"> V15.V25 </th>
   <th style="text-align:left;"> V16.V26 </th>
   <th style="text-align:left;"> V17.V27 </th>
   <th style="text-align:left;"> V1 </th>
   <th style="text-align:left;"> V2 </th>
   <th style="text-align:left;"> V3 </th>
   <th style="text-align:left;"> V4 </th>
   <th style="text-align:left;"> V5 </th>
   <th style="text-align:left;"> V11 </th>
   <th style="text-align:left;"> V12 </th>
   <th style="text-align:left;"> V13 </th>
   <th style="text-align:left;"> V14 </th>
   <th style="text-align:left;"> V15 </th>
   <th style="text-align:left;"> V16 </th>
   <th style="text-align:left;"> V17 </th>
   <th style="text-align:left;"> V21 </th>
   <th style="text-align:left;"> V22 </th>
   <th style="text-align:left;"> V23 </th>
   <th style="text-align:left;"> V24 </th>
   <th style="text-align:left;"> V25 </th>
   <th style="text-align:left;"> V26 </th>
   <th style="text-align:left;"> V27 </th>
   <th style="text-align:left;"> X46 </th>
   <th style="text-align:left;"> X47 </th>
   <th style="text-align:left;"> X47i1 </th>
   <th style="text-align:left;"> X47i2 </th>
   <th style="text-align:left;"> X47i3 </th>
   <th style="text-align:left;"> X48 </th>
   <th style="text-align:left;"> X49.65 </th>
   <th style="text-align:left;"> X49 </th>
   <th style="text-align:left;"> X49i1 </th>
   <th style="text-align:left;"> X50.64 </th>
   <th style="text-align:left;"> X50 </th>
   <th style="text-align:left;"> X50i1 </th>
   <th style="text-align:left;"> X50i2 </th>
   <th style="text-align:left;"> X50i3 </th>
   <th style="text-align:left;"> X50i4 </th>
   <th style="text-align:left;"> X50i5 </th>
   <th style="text-align:left;"> X50i6 </th>
   <th style="text-align:left;"> X50i7 </th>
   <th style="text-align:left;"> X51.63 </th>
   <th style="text-align:left;"> X51 </th>
   <th style="text-align:left;"> X51i1 </th>
   <th style="text-align:left;"> X51i2 </th>
   <th style="text-align:left;"> X51i3 </th>
   <th style="text-align:left;"> X52.62 </th>
   <th style="text-align:left;"> X52 </th>
   <th style="text-align:left;"> X52i1 </th>
   <th style="text-align:left;"> X53.61 </th>
   <th style="text-align:left;"> X53 </th>
   <th style="text-align:left;"> X53i1 </th>
   <th style="text-align:left;"> X54 </th>
   <th style="text-align:left;"> X54.58 </th>
   <th style="text-align:left;"> X54i1 </th>
   <th style="text-align:left;"> X54i2 </th>
   <th style="text-align:left;"> X54i3 </th>
   <th style="text-align:left;"> X54i4 </th>
   <th style="text-align:left;"> X55 </th>
   <th style="text-align:left;"> X55i1 </th>
   <th style="text-align:left;"> X55i2 </th>
   <th style="text-align:left;"> X55i3 </th>
   <th style="text-align:left;"> X55i4 </th>
   <th style="text-align:left;"> X55i5 </th>
   <th style="text-align:left;"> X56 </th>
   <th style="text-align:left;"> X56i1 </th>
   <th style="text-align:left;"> X56i2 </th>
   <th style="text-align:left;"> X56i3 </th>
   <th style="text-align:left;"> X56i4 </th>
   <th style="text-align:left;"> X57 </th>
   <th style="text-align:left;"> X57i1 </th>
   <th style="text-align:left;"> X58 </th>
   <th style="text-align:left;"> X58i1 </th>
   <th style="text-align:left;"> X58i2 </th>
   <th style="text-align:left;"> X59 </th>
   <th style="text-align:left;"> X59i1 </th>
   <th style="text-align:left;"> X60 </th>
   <th style="text-align:left;"> X60i1 </th>
   <th style="text-align:left;"> X60i2 </th>
   <th style="text-align:left;"> X60i3 </th>
   <th style="text-align:left;"> X60i4 </th>
   <th style="text-align:left;"> X60i5 </th>
   <th style="text-align:left;"> X60i6 </th>
   <th style="text-align:left;"> X60i7 </th>
   <th style="text-align:left;"> X60i8 </th>
   <th style="text-align:left;"> X60i9 </th>
   <th style="text-align:left;"> X60i10 </th>
   <th style="text-align:left;"> X60i11 </th>
   <th style="text-align:left;"> X61 </th>
   <th style="text-align:left;"> X61i1 </th>
   <th style="text-align:left;"> X62 </th>
   <th style="text-align:left;"> X63 </th>
   <th style="text-align:left;"> X64 </th>
   <th style="text-align:left;"> X64i1 </th>
   <th style="text-align:left;"> X64i2 </th>
   <th style="text-align:left;"> X65 </th>
   <th style="text-align:left;"> X65i1 </th>
   <th style="text-align:left;"> X65i2 </th>
   <th style="text-align:left;"> X65i3 </th>
   <th style="text-align:left;"> X65i4 </th>
   <th style="text-align:left;"> X65i5 </th>
   <th style="text-align:left;"> X65i6 </th>
   <th style="text-align:left;"> X65i7 </th>
   <th style="text-align:left;"> X65i8 </th>
   <th style="text-align:left;"> X65i9 </th>
   <th style="text-align:left;"> X66 </th>
   <th style="text-align:left;"> X66i1 </th>
   <th style="text-align:left;"> X67 </th>
   <th style="text-align:left;"> X67i1 </th>
   <th style="text-align:left;"> X68 </th>
   <th style="text-align:left;"> X68i1 </th>
   <th style="text-align:left;"> X68i2 </th>
   <th style="text-align:left;"> X69 </th>
   <th style="text-align:left;"> X69i1 </th>
   <th style="text-align:left;"> X70 </th>
   <th style="text-align:left;"> X70i1 </th>
   <th style="text-align:left;"> X70i2 </th>
   <th style="text-align:left;"> X70i3 </th>
   <th style="text-align:left;"> X70i4 </th>
   <th style="text-align:left;"> X70i5 </th>
   <th style="text-align:left;"> X70i6 </th>
   <th style="text-align:left;"> X70i7 </th>
   <th style="text-align:left;"> X70i8 </th>
   <th style="text-align:left;"> X70i9 </th>
   <th style="text-align:left;"> X71 </th>
   <th style="text-align:left;"> X71i1 </th>
   <th style="text-align:left;"> X71i2 </th>
   <th style="text-align:left;"> X72 </th>
   <th style="text-align:left;"> X73 </th>
   <th style="text-align:left;"> X74 </th>
   <th style="text-align:left;"> X75 </th>
   <th style="text-align:left;"> X76 </th>
  </tr>
 </thead>
<tbody>
  <tr>
   <td style="text-align:left;"> Nematoda </td>
   <td style="text-align:left;"> eukaryota </td>
   <td style="text-align:left;"> Cys </td>
   <td style="text-align:left;"> panRed1_KB455139.1.trna1-CysGCA </td>
   <td style="text-align:left;"> panRed1 </td>
   <td style="text-align:left;"> Panagrellus redivivus MT8872 </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> Cys </td>
   <td style="text-align:left;"> GCA </td>
   <td style="text-align:right;"> 69.8 </td>
   <td style="text-align:right;"> 91.5 </td>
   <td style="text-align:right;"> 0.4146341 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:right;"> 7 </td>
   <td style="text-align:right;"> 7 </td>
   <td style="text-align:right;"> 8 </td>
   <td style="text-align:right;"> 0 </td>
   <td style="text-align:left;"> TRUE </td>
   <td style="text-align:left;"> FALSE </td>
   <td style="text-align:left;"> G:C </td>
   <td style="text-align:left;"> G </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> G:C </td>
   <td style="text-align:left;"> G </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> G:C </td>
   <td style="text-align:left;"> G </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> U:G </td>
   <td style="text-align:left;"> U </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> C:G </td>
   <td style="text-align:left;"> C </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> U:G </td>
   <td style="text-align:left;"> U </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> A:U </td>
   <td style="text-align:left;"> A </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> U </td>
   <td style="text-align:left;"> U:A:A </td>
   <td style="text-align:left;"> U:A </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> A </td>
   <td style="text-align:left;"> A:U:A </td>
   <td style="text-align:left;"> A:A </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> G:C </td>
   <td style="text-align:left;"> G </td>
   <td style="text-align:left;"> G:C:G </td>
   <td style="text-align:left;"> G:G </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> C:G </td>
   <td style="text-align:left;"> C </td>
   <td style="text-align:left;"> U:A </td>
   <td style="text-align:left;"> U </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> C:G </td>
   <td style="text-align:left;"> C </td>
   <td style="text-align:left;"> C:G:G </td>
   <td style="text-align:left;"> A </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> G </td>
   <td style="text-align:left;"> G:C </td>
   <td style="text-align:left;"> U </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> G </td>
   <td style="text-align:left;"> G:U </td>
   <td style="text-align:left;"> G </td>
   <td style="text-align:left;"> G:C </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> C </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> A </td>
   <td style="text-align:left;"> G </td>
   <td style="text-align:left;"> G:G </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> A </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> G </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> C </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> A </td>
   <td style="text-align:left;"> A:A </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> A:U </td>
   <td style="text-align:left;"> A </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> U:A </td>
   <td style="text-align:left;"> U </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> C:G </td>
   <td style="text-align:left;"> C </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> G:C </td>
   <td style="text-align:left;"> G </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> G:C </td>
   <td style="text-align:left;"> G </td>
   <td style="text-align:left;"> C </td>
   <td style="text-align:left;"> U </td>
   <td style="text-align:left;"> G </td>
   <td style="text-align:left;"> C </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> A </td>
   <td style="text-align:left;"> G </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> U </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> C </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> C </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> G </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> A </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> U </td>
   <td style="text-align:left;"> A </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> G </td>
   <td style="text-align:left;"> -:- </td>
   <td style="text-align:left;"> -:- </td>
   <td style="text-align:left;"> -:- </td>
   <td style="text-align:left;"> -:- </td>
   <td style="text-align:left;"> -:- </td>
   <td style="text-align:left;"> -:- </td>
   <td style="text-align:left;"> -:- </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> G </td>
   <td style="text-align:left;"> U </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> C </td>
   <td style="text-align:left;"> C:G </td>
   <td style="text-align:left;"> C </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> C:G </td>
   <td style="text-align:left;"> C </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> C:G </td>
   <td style="text-align:left;"> C </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> G:C </td>
   <td style="text-align:left;"> G </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> G:C </td>
   <td style="text-align:left;"> G </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> U </td>
   <td style="text-align:left;"> U:A </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> U </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> C </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> A </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> A </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> C </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> U </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> C </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> C </td>
   <td style="text-align:left;"> G </td>
   <td style="text-align:left;"> G </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> G </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> U </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> G </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> G </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> G </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> C </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> C </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> . </td>
   <td style="text-align:left;"> C </td>
   <td style="text-align:left;"> U </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> - </td>
   <td style="text-align:left;"> - </td>
  </tr>
</tbody>
</table>

There are 5 copies of Cys in _Panagrellus redivivus_, a nematode commonly used to feed aquarium fish.

#### Asp-A46

In [116]:
## No. tRNAs without A46
identities %>% select(clade, species, isotype, X46) %>%
  filter(isotype == "Asp" & X46 != 'A') %>% nrow()

## Clade check
best_freqs %>% filter(positions == 'X46' & isotype == "Asp") %>% group_by(clade) %>%
  summarize(status=sum((variable == 'A')*count)/sum(count) == 1) %>%
  filter(!status)

## Species check
identities %>% select(clade, species, isotype, X46) %>%
  filter(isotype == "Asp") %>%
  group_by(clade, species) %>%
  summarize(status=sum(X46 == 'A')/n() >= 0.1,
            freq=sum(X46 == 'A'), tRNAs=n()) %>%
  filter(!status)

[1] 50

,clade,status


,clade,species,status,freq,tRNAs


Is this enriched in certain clades?

In [117]:
identities %>% select(clade, species, isotype, quality, X46) %>%
  filter(quality & isotype == "Asp" & X46 != 'A') %>% group_by(clade) %>% tally()

,clade,n
1,Mammalia,23
2,Nematoda,1
3,Spermatophyta,2


How about certain species? Pseudo tRNA counts are obtained from the quality set filter file.

In [154]:
sp = identities_all %>% select(species) %>% group_by(species) %>% tally() %>% filter(species %in% df$species) %>% select(n) %>% unlist

df = identities %>% select(clade, species, isotype, X46) %>%
  group_by(species) %>%
  mutate(ntRNAs = n()) %>%
  ungroup() %>%
  filter(isotype == "Asp" & X46 != 'A') %>% 
  group_by(clade, species, ntRNAs) %>% 
  summarize(countB46 = n()) %>%
  ungroup() %>%
  mutate(nPseudo = c(16835, 191918, 228656, 1908, 222374, 102, 1679, 93827, 244, 154651, 103, 8, 103, 36, 12, 98))

df

,clade,species,ntRNAs,countB46,nPseudo
1,Insecta,aedAeg1,299,9,16835
2,Mammalia,balAcu1,265,8,191918
3,Mammalia,bosTau8,302,4,228656
4,Mammalia,macEug2,154,2,1908
5,Mammalia,oviAri3,284,1,222374
6,Mammalia,panTro4,224,1,102
7,Mammalia,sarHar1,143,2,1679
8,Mammalia,sorAra2,160,1,93827
9,Mammalia,tarSyr2,178,1,244
10,Mammalia,turTru2,260,7,154651


What's the typical score for these guys?

In [153]:
# Score for all identified species
identities %>% select(clade, species, isotype, X46, score, isoscore) %>%
  filter(isotype == "Asp", species %in% df$species) %>%
  group_by(X46) %>%
  summarize(score = mean(score), isoscore = mean(isoscore))

# Score for low pseudogene count species
identities %>% select(clade, species, isotype, quality, X46, score, isoscore) %>%
  filter(isotype == "Asp", species %in% df$species) %>%
  mutate(species = ifelse(species %in% c("panTro4", "tarSyr2", "ce11", "araTha1", "braOle1", "carPap1", "orySat7", "solPen1"), "Low pseudo", "Hi pseudo")) %>%
  group_by(species, X46) %>%
  summarize(score = mean(score), isoscore = mean(isoscore))

,X46,score,isoscore
1,A,63.604,96.7266666666667
2,C,56.1,55.8
3,G,61.9255319148936,77.968085106383


,species,X46,score,isoscore
1,Hi pseudo,A,63.5264705882353,104.411764705882
2,Hi pseudo,G,61.4058823529412,79.2235294117647
3,Low pseudo,A,63.6682926829268,90.3536585365854
4,Low pseudo,C,56.1,55.8
5,Low pseudo,G,63.2846153846154,74.6846153846154


These actually score fairly well. In high tRNA pseudogene species, there's a slight difference between A46 and G46 Asp-specific scores, whereas in low pseudogene species, there's a negligible difference.

To be fair, there aren't a lot of low pseudogene species and tRNAs to compare. It's a small signal, a very small piece of evidence. I'll run the low pseudo guys through the single tRNA analysis pipeline.

In [156]:
identities %>%
  filter(isotype == "Asp", X46 == "G") %>% 
  filter(species %in% c("panTro4", "tarSyr2", "ce11", "araTha1", "braOle1", "carPap1", "orySat7", "solPen1")) %>%
  kable %>% as.character %>% display_html

clade,domain,isotype,seqname,species,species_long,taxid,quality,restrict,isotype_best,anticodon,score,isoscore,GC,intron,insertions,deletions,D.loop,AC.loop,TPC.loop,V.arm,X1.72,X1,X1i1,X2.71,X2,X2i1,X3.70,X3,X3i1,X3i2,X3i3,X3i4,X3i5,X3i6,X4.69,X4,X4i1,X4i2,X4i3,X4i4,X4i5,X4i6,X4i7,X4i8,X4i9,X4i10,X4i11,X5.68,X5,X5i1,X5i2,X5i3,X5i4,X5i5,X5i6,X6.67,X6,X6i1,X7.66,X7,X7i1,X7i2,X7i3,X7i4,X7i5,X7i6,X7i7,X7i8,X7i9,X8,X8.14.21,X8.14,X8i1,X8i2,X9,X9.12.23,X9.23,X9i1,X10.25,X10,X10.25.45,X10.45,X10i1,X11.24,X11,X12.23,X12,X12i1,X13.22,X13,X13.22.46,X14,X14i1,X14i2,X14i3,X14i4,X14i5,X14i6,X14i7,X14i8,X14i9,X14i10,X14i11,X14i12,X14i13,X15,X15.48,X16,X16i1,X16i2,X16i3,X16i4,X16i5,X16i6,X16i7,X16i8,X16i9,X16i10,X16i11,X16i12,X16i13,X16i14,X16i15,X16i16,X17,X17a,X18,X18.55,X19,X19.56,X19i1,X19i2,X19i3,X19i4,X19i5,X19i6,X19i7,X19i8,X19i9,X19i10,X20,X20i1,X20i2,X20i3,X20i4,X20a,X20b,X21,X22,X22.46,X22i1,X23,X23i1,X23i2,X23i3,X23i4,X23i5,X23i6,X24,X24i1,X25,X25i1,X25i2,X25i3,X25i4,X26,X26.44,X26i1,X26i2,X26i3,X26i4,X26i5,X26i6,X26i7,X26i8,X26i9,X26i10,X27.43,X27,X27i1,X27i2,X27i3,X27i4,X28.42,X28,X28i1,X29.41,X29,X29i1,X29i2,X30.40,X30,X30i1,X31.39,X31,X32,X33,X34,X35,X35i1,X36,X37,X37i1,X37i2,X37i3,X37i4,X37i5,X37i6,X37i7,X37i8,X37i9,X38,X38i1,X39,X39i1,X39i2,X40,X40i1,X40i2,X41,X41i1,X41i2,X42,X42i1,X43,X44,X44i1,X44i2,X44i3,X44i4,X44i5,X44i6,X44i7,X44i8,X44i9,X44i10,X44i11,X44i12,X44i13,X44i14,X44i15,X44i16,X44i17,X44i18,X44i19,X44i20,X44i21,X44i22,X44i23,X45,V11.V21,V12.V22,V13.V23,V14.V24,V15.V25,V16.V26,V17.V27,V1,V2,V3,V4,V5,V11,V12,V13,V14,V15,V16,V17,V21,V22,V23,V24,V25,V26,V27,X46,X47,X47i1,X47i2,X47i3,X48,X49.65,X49,X49i1,X50.64,X50,X50i1,X50i2,X50i3,X50i4,X50i5,X50i6,X50i7,X51.63,X51,X51i1,X51i2,X51i3,X52.62,X52,X52i1,X53.61,X53,X53i1,X54,X54.58,X54i1,X54i2,X54i3,X54i4,X55,X55i1,X55i2,X55i3,X55i4,X55i5,X56,X56i1,X56i2,X56i3,X56i4,X57,X57i1,X58,X58i1,X58i2,X59,X59i1,X60,X60i1,X60i2,X60i3,X60i4,X60i5,X60i6,X60i7,X60i8,X60i9,X60i10,X60i11,X61,X61i1,X62,X63,X64,X64i1,X64i2,X65,X65i1,X65i2,X65i3,X65i4,X65i5,X65i6,X65i7,X65i8,X65i9,X66,X66i1,X67,X67i1,X68,X68i1,X68i2,X69,X69i1,X70,X70i1,X70i2,X70i3,X70i4,X70i5,X70i6,X70i7,X70i8,X70i9,X71,X71i1,X71i2,X72,X73,X74,X75,X76
Spermatophyta,eukaryota,Asp,araTha1_chr1.trna30-AspGTC,araTha1,Arabidopsis thaliana (TAIR10 Feb 2011),.,TRUE,FALSE,Asp,GTC,71.4,92.5,0.3902439,0,0,1,8,7,8,0,G:C,G,.,U:G,U,.,C:G,C,.,.,.,.,.,.,G:C,G,.,.,.,.,.,.,.,.,.,.,.,U:A,U,.,.,.,.,.,.,U:A,U,.,G:C,G,.,.,.,.,.,.,.,.,.,U,U:A:A,U:A,.,.,A,A:A:U,A:U,.,G:U,G,G:U:G,G:G,.,U:A,U,A:U,A,.,U:G,U,U:G:G,A,.,.,.,.,.,.,.,.,.,.,.,.,.,G,G:C,U,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,-,-,G,G:U,G,G:C,.,.,.,.,.,.,.,.,.,.,U,.,.,.,.,A,-,A,G,G:G,.,U,.,.,.,.,.,.,A,.,U,.,.,.,.,U,U:U,.,.,.,.,.,.,.,.,.,.,C:G,C,.,.,.,.,C:G,C,.,C:G,C,.,.,G:C,G,.,C:G,C,C,U,G,U,.,C,A,.,.,.,.,.,.,.,.,.,C,.,G,.,.,C,.,.,G,.,.,G,.,G,U,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,G,-:-,-:-,-:-,-:-,-:-,-:-,-:-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,G,-,.,.,.,C,C:G,C,.,C:G,C,.,.,.,.,.,.,.,G:C,G,.,.,.,G:C,G,.,G:C,G,.,U,U:A,.,.,.,.,U,.,.,.,.,.,C,.,.,.,.,G,.,A,.,.,U,.,C,.,.,.,.,.,.,.,.,.,.,.,C,.,C,C,G,.,.,G,.,.,.,.,.,.,.,.,.,C,.,A,.,A,.,.,C,.,G,.,.,.,.,.,.,.,.,.,G,.,.,C,G,-,-,-
Spermatophyta,eukaryota,Asp,braOle1_NW_013618197.1.trna3-AspGTC,braOle1,Brassica oleracea var. oleracea TO1000,.,FALSE,FALSE,Ile,GTC,62.4,63.3,0.3902439,0,0,0,9,7,8,0,G:C,G,.,G:C,G,.,G:C,G,.,.,.,.,.,.,A:U,A,.,.,.,.,.,.,.,.,.,.,.,U:G,U,.,.,.,.,.,.,U:A,U,.,G:C,G,.,.,.,.,.,.,.,.,.,U,U:A:A,U:A,.,.,A,A:U:A,A:A,.,G:C,G,G:C:A,G:A,.,U:G,U,U:A,U,.,C:G,C,C:G:G,A,.,.,.,.,.,.,.,.,.,.,.,.,.,A,A:U,U,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,U,-,G,G:U,G,G:C,.,.,.,.,.,.,.,.,.,.,U,.,.,.,.,C,-,A,G,G:G,.,A,.,.,.,.,.,.,G,.,C,.,.,.,.,A,A:A,.,.,.,.,.,.,.,.,.,.,C:G,C,.,.,.,.,C:G,C,.,G:C,G,.,.,C:G,C,.,C:G,C,C,U,G,U,.,C,A,.,.,.,.,.,.,.,.,.,A,.,G,.,.,G,.,.,C,.,.,G,.,G,A,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,A,-:-,-:-,-:-,-:-,-:-,-:-,-:-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,G,C,.,.,.,U,G:U,G,.,C:G,C,.,.,.,.,.,.,.,G:C,G,.,.,.,G:C,G,.,G:C,G,.,U

- _A. thaliana_'s Asp is the highest scoring one, and is single copy. It differs from all other Asp tRNAs by one base (G46). Same with _C. elegans_.
- In _O. sativa_, _T. syrichta_, and _P. troglodytes_, it's single copy.

Here are the bitcharts for these guys with respect to mammalian Asp: 
![Asp-G46 bitchart](figures/Asp-G46.png)